#Parquet Version of BacktestEngine 


In [1]:
#CLAUDE VERSION 
#version 2 

# AlgoHaus Backtester v5.0 - CustomTkinter Dark Edition (PARQUET VERSION)
# Features: Dark Theme UI with Curved Corners, Collapsible Trade Logs, Advanced Metrics, AI Analysis
# Modified to read PARQUET files instead of CSV files

import customtkinter as ctk
import tkinter as tk
from tkinter import messagebox, filedialog
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import webbrowser
import os
import tempfile
import inspect
import pathlib
import threading
import queue
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import re 
import logging

# Configure CustomTkinter
ctk.set_appearance_mode("dark")  # Modes: "System", "Dark", "Light"
ctk.set_default_color_theme("dark-blue")  # Themes: "blue", "green", "dark-blue"

# Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# ======================================================================
# 1. MARGIN AND PIP CALCULATIONS (Same as original)
# ======================================================================
class ForexCalculator:
    """Handle all forex calculations including margins, pip values, and position sizing"""
   
    # Leverage options
    LEVERAGE_OPTIONS = [1, 10, 20, 30, 50, 100, 200, 500]
   
    # Standard pip values (for 1 standard lot = 100,000 units)
    PIP_VALUES = {
        'EUR/USD': 0.0001, 'GBP/USD': 0.0001, 'USD/JPY': 0.01,
        'USD/CHF': 0.0001, 'USD/CAD': 0.0001, 'AUD/USD': 0.0001,
        'NZD/USD': 0.0001, 'EUR/JPY': 0.01, 'GBP/JPY': 0.01,
        'AUD/JPY': 0.01, 'EUR/GBP': 0.0001, 'EUR/CHF': 0.0001,
        'GBP/CHF': 0.0001, 'CHF/JPY': 0.01, 'CAD/JPY': 0.01
    }
    
    @staticmethod
    def get_max_units_per_100usd(leverage=50):
        """Calculates the maximum tradeable unit size for every $100 in the account."""
        return 100 * leverage 
   
    @staticmethod
    def calculate_pip_value_in_usd(pair, unit_size, current_price, conversion_rate=1.0):
        """Calculate pip value in USD for given pair and unit size."""
        pip_size = ForexCalculator.PIP_VALUES.get(pair, 0.0001) 
       
        if pair.endswith('/USD'):
            pip_value = pip_size * unit_size
        elif pair.startswith('USD/'):
            pip_value = (pip_size / current_price) * unit_size
        else:
            pip_value_in_base = pip_size * unit_size
            pip_value = pip_value_in_base * conversion_rate 
       
        return pip_value
   
    @staticmethod
    def calculate_margin_required(pair, unit_size, current_price, leverage, conversion_rate=1.0):
        """Calculate margin required for a position in USD."""
        if pair.endswith('/USD'):
            position_value = unit_size * current_price
        elif pair.startswith('USD/'):
            position_value = unit_size
        else:
            position_value = unit_size * conversion_rate 
       
        margin_required = position_value / leverage
        return margin_required
   
    @staticmethod
    def calculate_position_size(account_balance, risk_percent, stop_loss_pips, pair, current_price, conversion_rate=1.0):
        """Calculate optimal position size based on risk management"""
        risk_amount = account_balance * (risk_percent / 100)
        pip_value_per_unit = ForexCalculator.calculate_pip_value_in_usd(pair, 1, current_price, conversion_rate)
       
        if pip_value_per_unit > 0:
            position_size = risk_amount / (stop_loss_pips * pip_value_per_unit)
        else:
            position_size = 0
           
        return int(position_size) 

# ======================================================================
# 2. DATA LOADING FUNCTIONS - MODIFIED FOR PARQUET FILES
# ======================================================================

def get_pair_date_range(pair_name: str, base_folder: pathlib.Path):
    """Get the available date range for a pair without loading all data"""
    try:
        pair_clean = pair_name.replace('/', '_')
        subfolder = base_folder / pair_clean
        parquet_files = []
       
        if subfolder.is_dir():
            parquet_files = list(subfolder.glob("*.parquet"))
       
        if not parquet_files:
            parquet_files = [f for f in base_folder.glob("*.parquet") if pair_clean.lower() in f.name.lower()]
       
        if not parquet_files:
            return None, None
       
        parquet_path = parquet_files[0]
       
        # Parquet files are efficient for reading metadata
        # Read just the first and last few rows to get date range
        df_sample = pd.read_parquet(parquet_path, engine='pyarrow')
        
        if df_sample.empty:
            return None, None
           
        # Get the datetime column (first column or look for datetime-like column)
        datetime_col = None
        for col in df_sample.columns:
            if 'date' in col.lower() or 'time' in col.lower():
                datetime_col = col
                break
        if datetime_col is None:
            datetime_col = df_sample.columns[0]
            
        df_sample[datetime_col] = pd.to_datetime(df_sample[datetime_col], errors='coerce', utc=True)
        df_sample = df_sample.dropna(subset=[datetime_col])
       
        if df_sample.empty:
            return None, None
           
        first_date = df_sample[datetime_col].min()
        last_date = df_sample[datetime_col].max()
       
        if pd.isna(first_date) or pd.isna(last_date):
            return None, None
           
        return first_date.date(), last_date.date()
    except Exception as e:
        logging.error(f"Error getting date range for {pair_name}: {e}")
        return None, None

def load_pair_data(pair_name: str, base_folder: pathlib.Path, start_date: datetime, end_date: datetime, timeframe: str):
    """Load and optimize data as DataFrame for fast backtesting - PARQUET VERSION."""
    pair_clean = pair_name.replace('/', '_')
    subfolder = base_folder / pair_clean
    parquet_files = []
   
    if subfolder.is_dir():
        parquet_files = list(subfolder.glob("*.parquet"))
   
    if not parquet_files:
        parquet_files = [f for f in base_folder.glob("*.parquet") if pair_clean.lower() in f.name.lower()]
   
    if not parquet_files:
        raise FileNotFoundError(f"No PARQUET file found for {pair_name}")
   
    parquet_path = parquet_files[0]
    logging.info(f"Loading parquet file: {parquet_path}")
   
    # Read the entire parquet file (parquet is already optimized for columnar storage)
    df = pd.read_parquet(parquet_path, engine='pyarrow')
    
    if df.empty:
        raise ValueError("PARQUET file is empty")
   
    # Column mapping - detect column names
    cols_lower = [c.strip().lower() for c in df.columns]
    col_map = {
        'datetime': ['datetime', 'date', 'time', 'timestamp', 'date_time', 'index', 'tick'],
        'open': ['open', 'o', 'bid', 'ask', 'price'],
        'high': ['high', 'h', 'max'],
        'low': ['low', 'l', 'min'],
        'close': ['close', 'c', 'last', 'price'],
        'volume': ['volume', 'vol', 'v', 'tick_volume', 'size']
    }
   
    rename = {}
    for target, aliases in col_map.items():
        for alias in aliases:
            if any(alias in col for col in cols_lower):
                orig = next(col for col in df.columns if alias in col.lower())
                rename[orig] = target
                break
        else:
            if target != 'volume':
                raise KeyError(f"Column for '{target}' not found")
   
    # Rename columns to standard names
    df = df.rename(columns=rename)
   
    if 'volume' not in df.columns:
        df['volume'] = 1000
   
    # Convert data types for efficiency
    for col in ['open', 'high', 'low', 'close', 'volume']:
        if col in df.columns:
            df[col] = df[col].astype('float32')
   
    # Parse dates
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', utc=True)
    df = df.dropna(subset=['datetime'])
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    
    # Handle duplicate datetimes
    df = df.drop_duplicates(subset='datetime', keep='first')
    df = df.sort_values('datetime')
   
    # Get actual range
    actual_start = df['datetime'].min().date()
    actual_end = df['datetime'].max().date()
   
    # Filter to date range
    df = df.set_index('datetime')
    user_start = max(pd.Timestamp(start_date.date()), df.index.min())
    user_end = min(pd.Timestamp(end_date.date()) + pd.Timedelta(hours=23, minutes=59, seconds=59), df.index.max())
    df = df.loc[user_start:user_end].copy()
   
    if df.empty:
        raise ValueError(f"No data in range {start_date.date()} to {end_date.date()}")
   
    # Resample if needed
    if timeframe != '1min':
        rule = {'5min': '5T', '15min': '15T', '1hr': '1H', '1Day': '1D'}.get(timeframe, '1T')
        df = df.resample(rule).agg({
            'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'
        }).dropna()
   
    # Add daily data WITHOUT LOOK-AHEAD BIAS
    df = df.reset_index()
    df['date'] = df['datetime'].dt.date
   
    # Calculate previous day's values (no look-ahead)
    daily = df.groupby('date').agg({
        'high': 'max', 'low': 'min', 'close': 'last'
    })
    daily.columns = ['day_high', 'day_low', 'day_close']
   
    # SHIFT to avoid look-ahead bias
    daily['prev_high'] = daily['day_high'].shift(1)
    daily['prev_low'] = daily['day_low'].shift(1)
    daily['prev_close'] = daily['day_close'].shift(1)
   
    daily = daily.reset_index()
    df = pd.merge(df, daily[['date', 'prev_high', 'prev_low', 'prev_close']], on='date', how='left')
    df[['prev_high', 'prev_low', 'prev_close']] = df[['prev_high', 'prev_low', 'prev_close']].ffill()
   
    logging.info(f"Loaded {len(df)} rows from parquet file")
    return df, actual_start, actual_end

class TradingStrategies:
    @staticmethod
    def opening_range_strategy(df, sl_pips, tp_pips, pip_value):
        """Opening Range Strategy - NO LOOK-AHEAD BIAS"""
        df = df.copy()
        trades = []
       
        # Group by day for daily opening range
        for date in df['date'].unique():
            day_data = df[df['date'] == date].reset_index(drop=True)
            if len(day_data) < 31: 
                continue
           
            # First 30 minutes (assuming 1-min data)
            opening_range = day_data.iloc[:30]
            or_high = opening_range['high'].max()
            or_low = opening_range['low'].min()
            or_mid = (or_high + or_low) / 2
           
            # Entry at bar 31 (after 30-min range)
            if len(day_data) > 30:
                entry_bar = day_data.iloc[30]
                signal = 'BUY' if entry_bar['close'] > or_mid else 'SELL'
               
                trades.append({
                    'datetime': entry_bar['datetime'],
                    'entry_price': entry_bar['close'],
                    'signal': signal,
                    'stop_loss': sl_pips * pip_value,
                    'take_profit': tp_pips * pip_value,
                    'day_data': day_data[31:].reset_index(drop=True)
                })
       
        return trades

    @staticmethod
    def vwap_crossover_strategy(df, sl_pips, tp_pips, pip_value):
        """VWAP Crossover - NO LOOK-AHEAD BIAS"""
        df = df.copy()
       
        # Calculate VWAP properly
        df['tpv'] = df['volume'] * (df['high'] + df['low'] + df['close']) / 3
        df['cumvol'] = df.groupby('date')['volume'].cumsum()
        df['cumtpv'] = df.groupby('date')['tpv'].cumsum()
        df['vwap'] = df['cumtpv'] / df['cumvol']
       
        # Detect crossovers using PREVIOUS bar's VWAP
        df['prev_close'] = df['close'].shift(1)
        df['prev_vwap'] = df['vwap'].shift(1)
       
        # Signal when price crosses VWAP
        df['signal'] = np.where(
            (df['prev_close'] <= df['prev_vwap']) & (df['close'] > df['vwap']), 'BUY',
            np.where((df['prev_close'] >= df['prev_vwap']) & (df['close'] < df['vwap']), 'SELL', None)
        )
       
        # Get trade entries
        entries = df[df['signal'].notna()].copy()
        trades = []
       
        for idx, row in entries.iterrows():
            remaining_data = df[df.index > idx].reset_index(drop=True)
            if len(remaining_data) > 0:
                trades.append({
                    'datetime': row['datetime'],
                    'entry_price': row['close'],
                    'signal': row['signal'],
                    'stop_loss': sl_pips * pip_value,
                    'take_profit': tp_pips * pip_value,
                    'day_data': remaining_data
                })
       
        return trades

    @staticmethod
    def pivot_point_reversal_strategy(df, sl_pips, tp_pips, pip_value):
        """Pivot Point Reversal - NO LOOK-AHEAD BIAS"""
        df = df.copy()
       
        # Calculate pivots using PREVIOUS day's data
        df['pivot'] = (df['prev_high'] + df['prev_low'] + df['prev_close']) / 3
        df['r1'] = 2 * df['pivot'] - df['prev_low']
        df['s1'] = 2 * df['pivot'] - df['prev_high']
       
        # Remove first day (no previous data)
        df = df[df['prev_high'].notna()].copy()
       
        trades = []
        # Define a small tolerance zone (e.g., 5 pips)
        bounce_zone = 5 * pip_value
       
        # Check each bar for pivot bounces
        for i in range(1, len(df)):
            current = df.iloc[i]
           
            # Buy signal at support S1
            if abs(current['close'] - current['s1']) < bounce_zone:
                remaining = df.iloc[i+1:].reset_index(drop=True)
                if len(remaining) > 0:
                    trades.append({
                        'datetime': current['datetime'],
                        'entry_price': current['close'],
                        'signal': 'BUY',
                        'stop_loss': sl_pips * pip_value,
                        'take_profit': tp_pips * pip_value,
                        'day_data': remaining
                    })
           
            # Sell signal at resistance R1
            elif abs(current['close'] - current['r1']) < bounce_zone:
                remaining = df.iloc[i+1:].reset_index(drop=True)
                if len(remaining) > 0:
                    trades.append({
                        'datetime': current['datetime'],
                        'entry_price': current['close'],
                        'signal': 'SELL',
                        'stop_loss': sl_pips * pip_value,
                        'take_profit': tp_pips * pip_value,
                        'day_data': remaining
                    })
       
        return trades

class EnhancedBacktester:
    def __init__(self, df, initial_balance=10000, unit_size=10000, pip_value=0.0001, leverage=50):
        self.df = df
        self.initial_balance = initial_balance
        self.unit_size = unit_size
        self.pip_value = pip_value
        self.leverage = leverage
        self.results = None
        self.balance_history = []
        
    def get_conversion_rate(self, pair_name, bar_data):
        """Retrieves the necessary Base/USD conversion rate for cross-pairs."""
        return 1.0 
            
    def run_backtest(self, strategy_func, sl_pips, tp_pips, pair_name):
        """Run backtest with proper margin and P&L calculations"""
        logging.info(f"Dataframe size: {len(self.df)} bars.")
        
        trades = strategy_func(self.df, sl_pips, tp_pips, self.pip_value)
        
        logging.info(f"Strategy generated: {len(trades)} potential trades.")
        if not trades:
            self.results = pd.DataFrame()
            return "No trades generated.", {}
            
        results = []
        current_balance = self.initial_balance
        trade_number = 1
       
        for t in trades:
            entry_price = t['entry_price']
            signal = t['signal']
            bars = t['day_data']
            
            # Conversion and Margin Calculation
            conversion_rate = self.get_conversion_rate(pair_name, {'datetime': t['datetime'], 'close': entry_price})
           
            margin_required = ForexCalculator.calculate_margin_required(
                pair_name, self.unit_size, entry_price, self.leverage, conversion_rate
            )
           
            # Skip trade if insufficient margin
            if margin_required > current_balance * 0.5:
                continue
           
            pip_value_usd = ForexCalculator.calculate_pip_value_in_usd(
                pair_name, self.unit_size, entry_price, conversion_rate
            )
           
            # Set stop loss and take profit levels
            if signal == 'BUY':
                stop_level = entry_price - (sl_pips * self.pip_value)
                take_level = entry_price + (tp_pips * self.pip_value)
            else:
                stop_level = entry_price + (sl_pips * self.pip_value)
                take_level = entry_price - (tp_pips * self.pip_value)
           
            # Find exit point
            exit_idx = len(bars) - 1
            exit_reason = 'Timeout'
            exit_price = bars.iloc[-1]['close'] if len(bars) > 0 else entry_price
           
            for i, bar in bars.iterrows():
                if signal == 'BUY':
                    if bar['low'] <= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['high'] >= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
                else:
                    if bar['high'] >= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['low'] <= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
           
            # Calculate P&L
            if signal == 'BUY':
                pips_pnl = (exit_price - entry_price) / self.pip_value
            else:
                pips_pnl = (entry_price - exit_price) / self.pip_value
           
            monetary_pnl = pips_pnl * pip_value_usd
           
            # Calculate time in trade
            entry_time = t['datetime']
            if exit_idx < len(bars):
                exit_time = bars.iloc[exit_idx]['datetime']
            else:
                exit_time = bars.iloc[-1]['datetime'] if len(bars) > 0 else entry_time
           
            time_in_trade = exit_time - entry_time
            hours_in_trade = time_in_trade.total_seconds() / 3600
           
            # Update balance
            current_balance += monetary_pnl
           
            results.append({
                'trade_number': f"{trade_number:05d}",
                'entry_time': entry_time,
                'exit_time': exit_time,
                'time_in_trade_hours': round(hours_in_trade, 2),
                'signal': signal,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'exit_reason': exit_reason,
                'pips_pnl': round(pips_pnl, 2),
                'monetary_pnl': round(monetary_pnl, 2),
                'unit_size': self.unit_size,
                'margin_used': round(margin_required, 2),
                'balance': round(current_balance, 2),
                'pip_value_usd': round(pip_value_usd, 4)
            })
           
            trade_number += 1
            self.balance_history.append(current_balance)
       
        self.results = pd.DataFrame(results)
       
        # Calculate metrics
        if not self.results.empty:
            total_pnl = self.results['monetary_pnl'].sum()
            total_pips = self.results['pips_pnl'].sum()
            win_rate = (self.results['pips_pnl'] > 0).mean() * 100
           
            summary = (f"TRADES: {len(self.results)}\n"
                      f"WIN RATE: {win_rate:.1f}%\n"
                      f"P&L: ${total_pnl:,.2f}\n"
                      f"PIPS: {total_pips:,.1f}\n"
                      f"FINAL BALANCE: ${current_balance:,.2f}")
           
            metrics = QuantStatsMetrics.calculate_all_metrics(
                self.results, self.initial_balance
            )
        else:
            summary = "No trades executed"
            metrics = {}
       
        return summary, metrics

class QuantStatsMetrics:
    @staticmethod
    def calculate_all_metrics(trades_df, initial_balance):
        """Calculate comprehensive QuantStats-style metrics"""
        if trades_df.empty:
            return {}
       
        # Basic metrics
        total_trades = len(trades_df)
        winning_trades = (trades_df['monetary_pnl'] > 0).sum()
        losing_trades = (trades_df['monetary_pnl'] < 0).sum()
        win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0
       
        # P&L metrics
        total_pnl = trades_df['monetary_pnl'].sum()
        total_pips = trades_df['pips_pnl'].sum()
        avg_win = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl'].mean() if winning_trades > 0 else 0
        avg_loss = abs(trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl'].mean()) if losing_trades > 0 else 0
        avg_win_pips = trades_df[trades_df['pips_pnl'] > 0]['pips_pnl'].mean() if winning_trades > 0 else 0
        avg_loss_pips = abs(trades_df[trades_df['pips_pnl'] < 0]['pips_pnl'].mean()) if losing_trades > 0 else 0
       
        # Best and worst trades
        best_trade = trades_df['monetary_pnl'].max()
        worst_trade = trades_df['monetary_pnl'].min()
        best_trade_pips = trades_df['pips_pnl'].max()
        worst_trade_pips = trades_df['pips_pnl'].min()
       
        # Profit factor
        gross_profit = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl'].sum()
        gross_loss = abs(trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl'].sum())
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
       
        # Returns
        final_balance = trades_df['balance'].iloc[-1] if not trades_df.empty else initial_balance
        total_return = ((final_balance - initial_balance) / initial_balance) * 100
       
        # Create equity curve
        equity_curve = initial_balance + trades_df['monetary_pnl'].cumsum()
       
        # Drawdown calculations
        cummax = equity_curve.expanding().max()
        drawdown = (equity_curve - cummax) / cummax
        max_drawdown_pct = drawdown.min() * 100
       
        # Time-based metrics
        start_date = trades_df['entry_time'].min()
        end_date = trades_df['exit_time'].max()
        trading_days = (end_date - start_date).days if pd.notna(start_date) and pd.notna(end_date) else 1
       
        # Annualized return
        years = trading_days / 365 if trading_days > 0 else 1
        annualized_return = ((final_balance / initial_balance) ** (1/years) - 1) * 100 if years > 0 else 0
       
        # Sharpe Ratio
        if len(trades_df) > 1:
            daily_returns = trades_df.groupby(trades_df['entry_time'].dt.date)['monetary_pnl'].sum()
            daily_returns_pct = daily_returns / initial_balance
            sharpe = (daily_returns_pct.mean() * 252) / (daily_returns_pct.std() * np.sqrt(252)) if daily_returns_pct.std() > 0 else 0
        else:
            sharpe = 0
       
        # Sortino Ratio
        if len(trades_df) > 1:
            negative_returns = daily_returns_pct[daily_returns_pct < 0]
            downside_std = negative_returns.std() if len(negative_returns) > 0 else 0
            sortino = (daily_returns_pct.mean() * 252) / (downside_std * np.sqrt(252)) if downside_std > 0 else 0
        else:
            sortino = 0
       
        # Average time in trade
        avg_time_in_trade = trades_df['time_in_trade_hours'].mean() if 'time_in_trade_hours' in trades_df else 0
       
        # Margin metrics
        avg_margin = trades_df['margin_used'].mean() if 'margin_used' in trades_df else 0
       
        # Average trade PnL
        avg_trade_pnl = total_pnl / total_trades if total_trades > 0 else 0
       
        return {
            # Trade Statistics
            'total_trades': total_trades,
            'winning_trades': winning_trades,
            'losing_trades': losing_trades,
            'win_rate_%': round(win_rate, 2),
           
            # P&L Metrics
            'total_pnl_$': round(total_pnl, 2),
            'total_pips': round(total_pips, 1),
            'avg_win_$': round(avg_win, 2),
            'avg_loss_$': round(avg_loss, 2),
            'avg_win_pips': round(avg_win_pips, 1),
            'avg_loss_pips': round(avg_loss_pips, 1),
            'avg_trade_pnl_$': round(avg_trade_pnl, 2),
           
            # Best/Worst
            'best_trade_$': round(best_trade, 2),
            'worst_trade_$': round(worst_trade, 2),
            'best_trade_pips': round(best_trade_pips, 1),
            'worst_trade_pips': round(worst_trade_pips, 1),
           
            # Risk Metrics
            'profit_factor': round(profit_factor, 2),
            'max_drawdown_%': round(max_drawdown_pct, 2),
            'sharpe_ratio': round(sharpe, 2),
            'sortino_ratio': round(sortino, 2),
           
            # Return Metrics
            'total_return_%': round(total_return, 2),
            'annualized_return_%': round(annualized_return, 2),
           
            # Other Metrics
            'avg_time_in_trade_hours': round(avg_time_in_trade, 2),
            'avg_margin_used_$': round(avg_margin, 2),
            'final_balance_$': round(final_balance, 2)
        }

# ======================================================================
# 6. ADVANCED HTML REPORT GENERATOR - ENHANCED VERSION
# ======================================================================
class AdvancedHTMLReportGenerator:
    @staticmethod
    def generate_dashboard_report(metrics, trades_df, strategy, timeframe, pair, initial_balance, unit_size, df=None):
        """Generate comprehensive HTML report with dashboard elements"""
       
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        strategy_code = AdvancedHTMLReportGenerator.get_strategy_code(strategy)
        comparison_chart_html = AdvancedHTMLReportGenerator.generate_comparison_chart(trades_df, initial_balance, df)
        plots_html = AdvancedHTMLReportGenerator.generate_plots(trades_df, initial_balance, df)
        advanced_charts_html = AdvancedHTMLReportGenerator.generate_advanced_charts(trades_df, initial_balance, metrics)
        analysis = AdvancedHTMLReportGenerator.generate_enhanced_ai_analysis(metrics, trades_df, df)
       
        html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>AlgoHaus Backtest Report - {pair} {strategy.__name__}</title>
            <meta charset="utf-8">
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            <style>
                /* FULL BLACK THEME */
                * {{
                    box-sizing: border-box;
                    margin: 0;
                    padding: 0;
                }}
                
                body {{
                    font-family: 'Helvetica', 'Courier New', monospace;
                    background: #000000;
                    color: #ffffff;
                    line-height: 1.6;
                }}
                
                .header {{
                    background: #000000;
                    color: #ffffff;
                    padding: 30px;
                    border-bottom: 2px solid #1976D2;
                    text-align: center;
                }}
                
                .header h1 {{
                    font-size: 24px;
                    margin-bottom: 10px;
                    color: #1976D2;
                    text-shadow: 0 0 10px rgba(25, 118, 210, 0.5);
                }}
                
                .header p {{
                    color: #888888;
                    font-size: 14px;
                }}
                
                .primary-chart {{
                    background: #000000;
                    padding: 20px;
                    border-bottom: 1px solid #222222;
                }}
                
                .primary-chart h2 {{
                    color: #1976D2;
                    font-size: 18px;
                    margin-bottom: 20px;
                    font-weight: bold;
                }}
                
                /* Strategy Code Section */
                .strategy-code-section {{
                    background: #000000;
                    padding: 20px;
                    border-bottom: 1px solid #222222;
                }}
                
                .strategy-code-button {{
                    background: linear-gradient(135deg, #1976D2, #0d47a1);
                    color: white;
                    border: none;
                    padding: 12px 24px;
                    font-size: 14px;
                    font-family: 'Helvetica', monospace;
                    cursor: pointer;
                    border-radius: 4px;
                    transition: all 0.3s ease;
                    box-shadow: 0 2px 5px rgba(25, 118, 210, 0.3);
                }}
                
                .strategy-code-button:hover {{
                    background: linear-gradient(135deg, #2196F3, #1976D2);
                    box-shadow: 0 4px 8px rgba(25, 118, 210, 0.5);
                    transform: translateY(-1px);
                }}
                
                .strategy-code-content {{
                    display: none;
                    background: #0a0a0a;
                    border: 1px solid #222222;
                    border-radius: 5px;
                    padding: 20px;
                    margin-top: 15px;
                    overflow-x: auto;
                }}
                
                .strategy-code-content.show {{
                    display: block;
                    animation: fadeIn 0.3s ease-in;
                }}
                
                @keyframes fadeIn {{
                    from {{ opacity: 0; transform: translateY(-10px); }}
                    to {{ opacity: 1; transform: translateY(0); }}
                }}
                
                .strategy-code-content pre {{
                    margin: 0;
                    color: #e0e0e0;
                    font-family: 'Helvetica', 'Courier New', monospace;
                    font-size: 12px;
                    line-height: 1.4;
                }}
                
                /* Code Syntax Highlighting */
                .code-keyword {{ color: #569CD6; font-weight: bold; }}
                .code-string {{ color: #CE9178; }}
                .code-comment {{ color: #6A9955; font-style: italic; }}
                .code-function {{ color: #DCDCAA; }}
                .code-number {{ color: #B5CEA8; }}
                
                /* Metrics Section */
                .metrics-section {{
                    background: #000000;
                    padding: 20px;
                    border-bottom: 1px solid #222222;
                }}
                
                .metrics-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
                    gap: 20px;
                    margin-top: 20px;
                }}
                
                .metric-card {{
                    background: #0a0a0a;
                    border: 1px solid #222222;
                    border-radius: 8px;
                    padding: 15px;
                    transition: all 0.3s ease;
                }}
                
                .metric-card:hover {{
                    background: #111111;
                    border-color: #1976D2;
                    box-shadow: 0 0 10px rgba(25, 118, 210, 0.2);
                }}
                
                .metric-label {{
                    color: #888888;
                    font-size: 12px;
                    text-transform: uppercase;
                    letter-spacing: 1px;
                    margin-bottom: 5px;
                }}
                
                .metric-value {{
                    font-size: 24px;
                    font-weight: bold;
                    color: #ffffff;
                }}
                
                .metric-value.positive {{ color: #4CAF50; }}
                .metric-value.negative {{ color: #F44336; }}
                .metric-value.neutral {{ color: #FFC107; }}
                
                /* Analysis Section */
                .analysis-section {{
                    background: #000000;
                    padding: 30px;
                    border-bottom: 1px solid #222222;
                }}
                
                .analysis-section h2 {{
                    color: #1976D2;
                    font-size: 18px;
                    margin-bottom: 20px;
                    display: flex;
                    align-items: center;
                }}
                
                .analysis-content {{
                    background: #0a0a0a;
                    border: 1px solid #222222;
                    border-radius: 8px;
                    padding: 25px;
                }}
                
                .analysis-text {{
                    line-height: 1.8;
                    font-size: 14px;
                    color: #e0e0e0;
                }}
                
                .analysis-section-title {{
                    color: #1976D2;
                    font-weight: bold;
                    margin-top: 20px;
                    margin-bottom: 10px;
                    font-size: 16px;
                    border-bottom: 1px solid #222222;
                    padding-bottom: 5px;
                }}
                
                .analysis-highlight {{
                    background: #111111;
                    padding: 15px;
                    border-left: 3px solid #1976D2;
                    margin: 15px 0;
                    border-radius: 3px;
                }}
                
                /* Collapsible Trade Log */
                .trades-section {{
                    background: #000000;
                    padding: 20px;
                    border-bottom: 1px solid #222222;
                }}
                
                .trades-header {{
                    display: flex;
                    justify-content: space-between;
                    align-items: center;
                    cursor: pointer;
                    padding: 15px;
                    background: #0a0a0a;
                    border: 1px solid #222222;
                    border-radius: 8px;
                    transition: all 0.3s ease;
                }}
                
                .trades-header:hover {{
                    background: #111111;
                    border-color: #1976D2;
                }}
                
                .trades-header h2 {{
                    color: #ffffff;
                    font-size: 16px;
                    margin: 0;
                    font-weight: bold;
                }}
                
                .toggle-icon {{
                    color: #1976D2;
                    font-size: 20px;
                    transition: transform 0.3s ease;
                }}
                
                .toggle-icon.rotated {{
                    transform: rotate(180deg);
                }}
                
                .trades-content {{
                    max-height: 0;
                    overflow: hidden;
                    transition: max-height 0.5s ease-out;
                }}
                
                .trades-content.expanded {{
                    max-height: 2000px;
                    transition: max-height 0.5s ease-in;
                }}
                
                .trades-table-container {{
                    padding: 20px;
                    background: #0a0a0a;
                    border: 1px solid #222222;
                    border-top: none;
                    border-radius: 0 0 8px 8px;
                    overflow-x: auto;
                }}
                
                /* Table Styling */
                table {{
                    width: 100%;
                    border-collapse: collapse;
                    font-family: 'Helvetica', monospace;
                    font-size: 12px;
                }}
                
                th {{
                    background: #111111;
                    color: #1976D2;
                    padding: 10px 6px;
                    text-align: left;
                    font-weight: bold;
                    border-bottom: 2px solid #1976D2;
                    white-space: nowrap;
                    position: sticky;
                    top: 0;
                    z-index: 10;
                }}
                
                td {{
                    padding: 8px 6px;
                    border-bottom: 1px solid #111111;
                    white-space: nowrap;
                    color: #e0e0e0;
                }}
                
                tr:nth-child(even) {{
                    background-color: #050505;
                }}
                
                tr:hover {{
                    background-color: #111111;
                    cursor: pointer;
                }}
                
                /* Trade Status Colors */
                .trade-win {{ color: #4CAF50; font-weight: bold; }}
                .trade-loss {{ color: #F44336; font-weight: bold; }}
                .trade-neutral {{ color: #FFC107; }}
                
                /* Charts Grid */
                .charts-container {{
                    background: #000000;
                    padding: 20px;
                }}
                
                .charts-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(500px, 1fr));
                    gap: 20px;
                }}
                
                .chart-box {{
                    background: #0a0a0a;
                    border: 1px solid #222222;
                    border-radius: 8px;
                    padding: 15px;
                }}
                
                .chart-box h3 {{
                    color: #1976D2;
                    font-size: 14px;
                    margin-bottom: 15px;
                    text-align: center;
                }}
                
                /* Responsive Design */
                @media (max-width: 768px) {{
                    .metrics-grid {{
                        grid-template-columns: 1fr;
                    }}
                    .charts-grid {{
                        grid-template-columns: 1fr;
                    }}
                }}
                
                /* Scrollbar Styling */
                ::-webkit-scrollbar {{
                    width: 10px;
                    height: 10px;
                }}
                
                ::-webkit-scrollbar-track {{
                    background: #0a0a0a;
                }}
                
                ::-webkit-scrollbar-thumb {{
                    background: #1976D2;
                    border-radius: 5px;
                }}
                
                ::-webkit-scrollbar-thumb:hover {{
                    background: #2196F3;
                }}
            </style>
        </head>
        <body>
        <div class="header">
            <h1>🚀 AlgoHaus Backtest Report v5.0 (PARQUET Edition)</h1>
            <p>📊 {pair} | 💡 {strategy.__name__} | ⏰ {timeframe} | 📅 {timestamp}</p>
        </div>
        
        <!-- Primary Comparison Chart -->
        <div class="primary-chart">
            <h2>📈 Equity Curve and Price Action Comparison</h2>
            {comparison_chart_html}
        </div>
        
        <!-- Metrics Dashboard -->
        <div class="metrics-section">
            <h2 style="color: #1976D2; font-size: 18px; margin-bottom: 0;">📊 Performance Metrics Dashboard</h2>
            {AdvancedHTMLReportGenerator.generate_metrics_dashboard(metrics, initial_balance)}
        </div>
        
        <!-- Enhanced AI Analysis -->
        <div class="analysis-section">
            <h2>🧠 Advanced Performance Analysis</h2>
            <div class="analysis-content">
                {analysis}
            </div>
        </div>
        
        <!-- Additional Charts -->
        <div class="charts-container">
            <h2 style="color: #1976D2; font-size: 18px; margin-bottom: 20px;">📊 Performance Visualizations</h2>
            <div class="charts-grid">
                {plots_html}
                {advanced_charts_html}
            </div>
        </div>
        
        <!-- Strategy Source Code -->
        <div class="strategy-code-section">
            <button class="strategy-code-button" onclick="toggleCode()">
                {'</>'} View Strategy Source Code
            </button>
            <div id="codeContent" class="strategy-code-content">
                <pre>{AdvancedHTMLReportGenerator.highlight_code(strategy_code)}</pre>
            </div>
        </div>
        
        <!-- Collapsible Trade Log -->
        <div class="trades-section">
            <div class="trades-header" onclick="toggleTrades()">
                <h2>📝 Detailed Trade Log ({len(trades_df)} Trades)</h2>
                <span class="toggle-icon" id="toggleIcon">▼</span>
            </div>
            <div class="trades-content" id="tradesContent">
                <div class="trades-table-container">
                    {AdvancedHTMLReportGenerator.generate_enhanced_trades_table(trades_df)}
                </div>
            </div>
        </div>
        
        <script>
            // Toggle Strategy Code
            function toggleCode() {{
                const codeContent = document.getElementById('codeContent');
                codeContent.classList.toggle('show');
            }}
            
            // Toggle Trade Log
            function toggleTrades() {{
                const tradesContent = document.getElementById('tradesContent');
                const toggleIcon = document.getElementById('toggleIcon');
                
                tradesContent.classList.toggle('expanded');
                toggleIcon.classList.toggle('rotated');
            }}
            
            // Auto-expand trade log if less than 20 trades
            window.onload = function() {{
                const tradeCount = {len(trades_df)};
                if (tradeCount <= 20) {{
                    document.getElementById('tradesContent').classList.add('expanded');
                    document.getElementById('toggleIcon').classList.add('rotated');
                }}
            }};
        </script>
        </body>
        </html>
        """
        
        temp_dir = tempfile.gettempdir()
        report_path = os.path.join(temp_dir, f"backtest_report_{pair.replace('/', '_')}_{strategy.__name__}_{timestamp}.html")
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(html)
            
        return report_path

    @staticmethod
    def generate_metrics_dashboard(metrics, initial_balance):
        """Generate a visual metrics dashboard with cards"""
        
        def get_metric_class(value, metric_type):
            if metric_type == 'return':
                return 'positive' if value >= 0 else 'negative'
            elif metric_type == 'winrate':
                return 'positive' if value >= 50 else 'negative'
            elif metric_type == 'factor':
                return 'positive' if value >= 1 else 'negative'
            elif metric_type == 'ratio':
                return 'positive' if value >= 0.5 else 'negative'
            elif metric_type == 'drawdown':
                return 'negative' if value < -10 else 'neutral'
            else:
                return 'neutral'
        
        html = f"""
        <div class="metrics-grid">
            <div class="metric-card">
                <div class="metric-label">Initial Balance</div>
                <div class="metric-value neutral">${initial_balance:,.2f}</div>
            </div>
            
            <div class="metric-card">
                <div class="metric-label">Final Balance</div>
                <div class="metric-value {get_metric_class(metrics.get('final_balance_$', 0) - initial_balance, 'return')}">${metrics.get('final_balance_$', 0):,.2f}</div>
            </div>
            
            <div class="metric-card">
                <div class="metric-label">Total Return</div>
                <div class="metric-value {get_metric_class(metrics.get('total_return_%', 0), 'return')}">{metrics.get('total_return_%', 0):+.2f}%</div>
            </div>
            
            <div class="metric-card">
                <div class="metric-label">Annualized Return</div>
                <div class="metric-value {get_metric_class(metrics.get('annualized_return_%', 0), 'return')}">{metrics.get('annualized_return_%', 0):+.2f}%</div>
            </div>
            
            <div class="metric-card">
                <div class="metric-label">Total Trades</div>
                <div class="metric-value neutral">{metrics.get('total_trades', 0)}</div>
            </div>
            
            <div class="metric-card">
                <div class="metric-label">Win Rate</div>
                <div class="metric-value {get_metric_class(metrics.get('win_rate_%', 0), 'winrate')}">{metrics.get('win_rate_%', 0):.1f}%</div>
            </div>
            
            <div class="metric-card">
                <div class="metric-label">Profit Factor</div>
                <div class="metric-value {get_metric_class(metrics.get('profit_factor', 0), 'factor')}">{metrics.get('profit_factor', 0):.2f}</div>
            </div>
            
            <div class="metric-card">
                <div class="metric-label">Max Drawdown</div>
                <div class="metric-value {get_metric_class(metrics.get('max_drawdown_%', 0), 'drawdown')}">{metrics.get('max_drawdown_%', 0):.2f}%</div>
            </div>
            
            <div class="metric-card">
                <div class="metric-label">Sharpe Ratio</div>
                <div class="metric-value {get_metric_class(metrics.get('sharpe_ratio', 0), 'ratio')}">{metrics.get('sharpe_ratio', 0):.2f}</div>
            </div>
            
            <div class="metric-card">
                <div class="metric-label">Sortino Ratio</div>
                <div class="metric-value {get_metric_class(metrics.get('sortino_ratio', 0), 'ratio')}">{metrics.get('sortino_ratio', 0):.2f}</div>
            </div>
            
            <div class="metric-card">
                <div class="metric-label">Total P&L</div>
                <div class="metric-value {get_metric_class(metrics.get('total_pnl_$', 0), 'return')}">${metrics.get('total_pnl_$', 0):+,.2f}</div>
            </div>
            
            <div class="metric-card">
                <div class="metric-label">Total Pips</div>
                <div class="metric-value {get_metric_class(metrics.get('total_pips', 0), 'return')}">{metrics.get('total_pips', 0):+,.1f}</div>
            </div>
        </div>
        """
        return html

    @staticmethod
    def generate_enhanced_trades_table(trades_df):
        """Generate enhanced trades table with better formatting"""
        if trades_df.empty:
            return "<p>No trades executed in the backtest.</p>"
        
        html = "<table>"
        html += """
        <thead>
            <tr>
                <th>#</th>
                <th>Entry Time</th>
                <th>Exit Time</th>
                <th>Duration (hrs)</th>
                <th>Signal</th>
                <th>Entry Price</th>
                <th>Exit Price</th>
                <th>Pips P&L</th>
                <th>$ P&L</th>
                <th>Exit Reason</th>
                <th>Balance</th>
            </tr>
        </thead>
        <tbody>
        """
        
        for idx, row in trades_df.iterrows():
            pnl_class = 'trade-win' if row['monetary_pnl'] > 0 else 'trade-loss' if row['monetary_pnl'] < 0 else 'trade-neutral'
            
            html += f"""
            <tr>
                <td>{row['trade_number']}</td>
                <td>{row['entry_time'].strftime('%Y-%m-%d %H:%M')}</td>
                <td>{row['exit_time'].strftime('%Y-%m-%d %H:%M')}</td>
                <td>{row['time_in_trade_hours']:.1f}</td>
                <td class="{pnl_class}">{row['signal']}</td>
                <td>{row['entry_price']:.5f}</td>
                <td>{row['exit_price']:.5f}</td>
                <td class="{pnl_class}">{row['pips_pnl']:+.1f}</td>
                <td class="{pnl_class}">${row['monetary_pnl']:+,.2f}</td>
                <td>{row['exit_reason']}</td>
                <td>${row['balance']:,.2f}</td>
            </tr>
            """
        
        html += "</tbody></table>"
        return html

    @staticmethod
    def generate_enhanced_ai_analysis(metrics, trades_df, df):
        """Generate enhanced AI-style analysis with detailed insights"""
        
        total_trades = metrics.get('total_trades', 0)
        win_rate = metrics.get('win_rate_%', 0)
        profit_factor = metrics.get('profit_factor', 0)
        sharpe = metrics.get('sharpe_ratio', 0)
        sortino = metrics.get('sortino_ratio', 0)
        max_dd = metrics.get('max_drawdown_%', 0)
        total_return = metrics.get('total_return_%', 0)
        ann_return = metrics.get('annualized_return_%', 0)
        
        # Performance Classification
        if sharpe >= 2 and profit_factor >= 1.5 and win_rate >= 50:
            performance = "EXCELLENT"
            emoji = "🌟"
        elif sharpe >= 1 and profit_factor >= 1.2:
            performance = "GOOD"
            emoji = "✅"
        elif profit_factor >= 1:
            performance = "ACCEPTABLE"
            emoji = "📊"
        else:
            performance = "POOR"
            emoji = "⚠️"
        
        analysis_html = f"""
        <div class="analysis-text">
            <div class="analysis-section-title">{emoji} Overall Performance: {performance}</div>
            
            <div class="analysis-highlight">
                <strong>Executive Summary:</strong><br>
                The strategy completed {total_trades} trades with a {win_rate:.1f}% win rate, generating a total return of {total_return:+.2f}% 
                ({ann_return:+.2f}% annualized). The risk-adjusted performance shows a Sharpe ratio of {sharpe:.2f} and Sortino ratio of {sortino:.2f}.
            </div>
            
            <div class="analysis-section-title">📈 Profitability Analysis</div>
            <p>
        """
        
        if profit_factor >= 1.5:
            analysis_html += f"<strong>Strong Profitability:</strong> The profit factor of {profit_factor:.2f} indicates that gross profits exceed gross losses by {(profit_factor-1)*100:.0f}%. This suggests a robust edge in the market.<br><br>"
        elif profit_factor >= 1:
            analysis_html += f"<strong>Positive Profitability:</strong> With a profit factor of {profit_factor:.2f}, the strategy is profitable but has limited margin for error. Consider optimizing entry/exit rules.<br><br>"
        else:
            analysis_html += f"<strong>Unprofitable Strategy:</strong> The profit factor of {profit_factor:.2f} indicates losses exceed profits. Major adjustments to the strategy logic are recommended.<br><br>"
        
        # Win Rate Analysis
        if win_rate >= 60:
            analysis_html += f"<strong>High Win Rate:</strong> At {win_rate:.1f}%, the strategy demonstrates excellent trade selection. Ensure proper position sizing to maximize this advantage.<br><br>"
        elif win_rate >= 45:
            analysis_html += f"<strong>Balanced Win Rate:</strong> The {win_rate:.1f}% win rate is within acceptable ranges. Focus on improving the reward-to-risk ratio for better overall performance.<br><br>"
        else:
            analysis_html += f"<strong>Low Win Rate:</strong> With only {win_rate:.1f}% winning trades, the strategy requires larger winning trades to compensate for frequent losses.<br><br>"
        
        analysis_html += f"""
            </p>
            
            <div class="analysis-section-title">⚡ Risk Management Assessment</div>
            <p>
        """
        
        # Drawdown Analysis
        if abs(max_dd) <= 10:
            analysis_html += f"<strong>Excellent Risk Control:</strong> Maximum drawdown of {max_dd:.2f}% shows exceptional capital preservation.<br><br>"
        elif abs(max_dd) <= 20:
            analysis_html += f"<strong>Acceptable Risk:</strong> The {max_dd:.2f}% maximum drawdown is within reasonable limits for most trading strategies.<br><br>"
        else:
            analysis_html += f"<strong>High Risk Warning:</strong> A maximum drawdown of {max_dd:.2f}% indicates significant capital risk. Consider implementing tighter stop-losses or reducing position sizes.<br><br>"
        
        # Sharpe/Sortino Analysis
        analysis_html += f"<strong>Risk-Adjusted Returns:</strong> "
        if sharpe >= 1.5:
            analysis_html += f"The Sharpe ratio of {sharpe:.2f} indicates excellent risk-adjusted returns. "
        elif sharpe >= 0.5:
            analysis_html += f"The Sharpe ratio of {sharpe:.2f} shows moderate risk-adjusted performance. "
        else:
            analysis_html += f"The low Sharpe ratio of {sharpe:.2f} suggests poor risk-adjusted returns. "
        
        if sortino > sharpe:
            analysis_html += f"The higher Sortino ratio ({sortino:.2f}) indicates that losses are less frequent than the volatility suggests, which is favorable."
        else:
            analysis_html += f"The Sortino ratio ({sortino:.2f}) confirms the risk assessment."
        
        analysis_html += """
            </p>
            
            <div class="analysis-section-title">🎯 Recommendations</div>
            <ul style="margin-left: 20px; color: #e0e0e0;">
        """
        
        # Generate specific recommendations
        recommendations = []
        
        if win_rate < 45:
            recommendations.append("• Improve entry signals: Consider adding confirmation indicators or tightening entry criteria")
        if abs(max_dd) > 20:
            recommendations.append("• Implement dynamic position sizing based on account equity to reduce drawdown")
        if profit_factor < 1.2:
            recommendations.append("• Optimize the risk-reward ratio: Consider wider take-profits or tighter stop-losses")
        if sharpe < 1:
            recommendations.append("• Reduce strategy volatility through better trade filtering or timing")
        if total_trades < 30:
            recommendations.append("• Limited sample size: Extend the backtesting period for more reliable statistics")
        
        if not recommendations:
            recommendations.append("• Strategy performing well - consider increasing position sizes gradually")
            recommendations.append("• Monitor live performance to ensure backtest results translate to real trading")
        
        for rec in recommendations:
            analysis_html += f"<li>{rec}</li>"
        
        # Time-based insights
        if not trades_df.empty:
            trades_df['hour'] = trades_df['entry_time'].dt.hour
            hourly_pnl = trades_df.groupby('hour')['monetary_pnl'].sum()
            best_hour = hourly_pnl.idxmax() if not hourly_pnl.empty else 0
            
            trades_df['weekday'] = trades_df['entry_time'].dt.day_name()
            daily_pnl = trades_df.groupby('weekday')['monetary_pnl'].sum()
            best_day = daily_pnl.idxmax() if not daily_pnl.empty else "N/A"
            
            analysis_html += f"""
            </ul>
            
            <div class="analysis-section-title">⏰ Temporal Patterns</div>
            <p>
                <strong>Best Trading Hour:</strong> {best_hour:02d}:00 UTC<br>
                <strong>Best Trading Day:</strong> {best_day}<br>
                <strong>Average Trade Duration:</strong> {metrics.get('avg_time_in_trade_hours', 0):.1f} hours
            </p>
            """
        
        analysis_html += "</div>"
        
        return analysis_html

    @staticmethod
    def generate_advanced_charts(trades_df, initial_balance, metrics):
        """Generate additional advanced charts"""
        if trades_df.empty:
            return ""
        
        html = ""
        
        # 1. Win/Loss Distribution
        wins = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl']
        losses = trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl']
        
        fig_dist = go.Figure()
        fig_dist.add_trace(go.Histogram(x=wins, name='Wins', marker_color='#4CAF50', opacity=0.7))
        fig_dist.add_trace(go.Histogram(x=losses, name='Losses', marker_color='#F44336', opacity=0.7))
        fig_dist.update_layout(
            template="plotly_dark",
            title="P&L Distribution",
            xaxis_title="P&L ($)",
            yaxis_title="Frequency",
            barmode='overlay',
            height=400,
            showlegend=True
        )
        html += f'<div class="chart-box"><h3>P&L Distribution</h3>{fig_dist.to_html(full_html=False, include_plotlyjs=False)}</div>'
        
        # 2. Cumulative P&L
        cumulative_pnl = trades_df['monetary_pnl'].cumsum()
        fig_cum = go.Figure()
        fig_cum.add_trace(go.Scatter(
            x=list(range(1, len(cumulative_pnl) + 1)),
            y=cumulative_pnl,
            mode='lines',
            line=dict(color='#1976D2', width=2),
            fill='tozeroy',
            fillcolor='rgba(25, 118, 210, 0.1)'
        ))
        fig_cum.update_layout(
            template="plotly_dark",
            title="Cumulative P&L",
            xaxis_title="Trade Number",
            yaxis_title="Cumulative P&L ($)",
            height=400
        )
        html += f'<div class="chart-box"><h3>Cumulative P&L</h3>{fig_cum.to_html(full_html=False, include_plotlyjs=False)}</div>'
        
        # 3. Monthly Returns (if enough data)
        if len(trades_df) > 30:
            trades_df['month'] = trades_df['entry_time'].dt.to_period('M')
            monthly_returns = trades_df.groupby('month')['monetary_pnl'].sum()
            
            if len(monthly_returns) > 1:
                fig_monthly = go.Figure(data=[
                    go.Bar(
                        x=[str(m) for m in monthly_returns.index],
                        y=monthly_returns.values,
                        marker_color=['#4CAF50' if x > 0 else '#F44336' for x in monthly_returns.values]
                    )
                ])
                fig_monthly.update_layout(
                    template="plotly_dark",
                    title="Monthly Returns",
                    xaxis_title="Month",
                    yaxis_title="P&L ($)",
                    height=400
                )
                html += f'<div class="chart-box"><h3>Monthly Returns</h3>{fig_monthly.to_html(full_html=False, include_plotlyjs=False)}</div>'
        
        # 4. Risk-Reward Scatter
        trades_df['risk_reward'] = abs(trades_df['monetary_pnl'] / trades_df['monetary_pnl'].std())
        fig_rr = go.Figure()
        fig_rr.add_trace(go.Scatter(
            x=list(range(1, len(trades_df) + 1)),
            y=trades_df['risk_reward'],
            mode='markers',
            marker=dict(
                color=trades_df['monetary_pnl'],
                colorscale='RdYlGn',
                size=8,
                showscale=True,
                colorbar=dict(title="P&L ($)")
            ),
            text=[f"Trade #{i}<br>P&L: ${pnl:.2f}" for i, pnl in enumerate(trades_df['monetary_pnl'], 1)],
            hovertemplate='%{text}<extra></extra>'
        ))
        fig_rr.update_layout(
            template="plotly_dark",
            title="Risk-Reward Profile",
            xaxis_title="Trade Number",
            yaxis_title="Risk-Reward Ratio",
            height=400
        )
        html += f'<div class="chart-box"><h3>Risk-Reward Profile</h3>{fig_rr.to_html(full_html=False, include_plotlyjs=False)}</div>'
        
        return html

    @staticmethod
    def generate_comparison_chart(trades_df, initial_balance, df):
        """Generates the combined equity curve and price chart using Plotly"""
        if trades_df.empty or df is None:
            return "<p>Not enough data or trades to generate the Comparison Chart.</p>"

        equity_curve = initial_balance + trades_df['monetary_pnl'].cumsum()
        
        max_eq_date = trades_df['exit_time'].max()
        df_plot = df[df['datetime'] <= max_eq_date]
        
        if df_plot.empty:
            return "<p>Not enough price data after trades to generate the Comparison Chart.</p>"
            
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                            vertical_spacing=0.1, 
                            row_heights=[0.7, 0.3])

        # Price Chart
        fig.add_trace(go.Candlestick(x=df_plot['datetime'],
                                     open=df_plot['open'],
                                     high=df_plot['high'],
                                     low=df_plot['low'],
                                     close=df_plot['close'],
                                     name='Price',
                                     increasing_line_color='#4CAF50', 
                                     decreasing_line_color='#F44336'), 
                      row=1, col=1)

        # Equity Curve
        fig.add_trace(go.Scatter(x=trades_df['exit_time'], y=equity_curve,
                                 mode='lines',
                                 name='Equity Curve',
                                 line=dict(color='#1976D2', width=2)), 
                      row=2, col=1)
                      
        # Trade markers
        winners = trades_df[trades_df['monetary_pnl'] > 0]
        losers = trades_df[trades_df['monetary_pnl'] < 0]

        fig.add_trace(go.Scatter(x=winners['exit_time'], y=winners['exit_price'],
                                 mode='markers', name='Win',
                                 marker=dict(symbol='triangle-up', size=8, color='#4CAF50'),
                                 hovertext='Win: $' + winners['monetary_pnl'].round(2).astype(str)),
                      row=1, col=1)
                      
        fig.add_trace(go.Scatter(x=losers['exit_time'], y=losers['exit_price'],
                                 mode='markers', name='Loss',
                                 marker=dict(symbol='triangle-down', size=8, color='#F44336'),
                                 hovertext='Loss: $' + losers['monetary_pnl'].round(2).astype(str)),
                      row=1, col=1)

        fig.update_layout(
            template="plotly_dark",
            title_text="",
            xaxis_rangeslider_visible=False,
            height=600,
            showlegend=False,
            paper_bgcolor='#000000',
            plot_bgcolor='#0a0a0a'
        )

        fig.update_yaxes(title_text="Price", row=1, col=1)
        fig.update_yaxes(title_text="Balance ($)", row=2, col=1)

        return fig.to_html(full_html=False, include_plotlyjs='cdn')

    @staticmethod
    def generate_plots(trades_df, initial_balance, df):
        """Generates additional metrics plots"""
        if trades_df.empty:
            return ""
            
        html = ""
        
        # Drawdown Chart
        equity_curve = initial_balance + trades_df['monetary_pnl'].cumsum()
        cummax = equity_curve.expanding().max()
        drawdown = (equity_curve - cummax) / cummax
        
        fig_dd = go.Figure(data=[
            go.Scatter(x=trades_df['exit_time'], y=drawdown * 100,
                       mode='lines', line=dict(color='#F44336', width=2),
                       fill='tozeroy', fillcolor='rgba(244, 67, 54, 0.3)')
        ])
        fig_dd.update_layout(
            template="plotly_dark", 
            title="Drawdown %", 
            yaxis_title="Drawdown (%)", 
            height=400,
            paper_bgcolor='#000000',
            plot_bgcolor='#0a0a0a'
        )
        html += f'<div class="chart-box"><h3>Drawdown Analysis</h3>{fig_dd.to_html(full_html=False, include_plotlyjs=False)}</div>'

        # P&L by Day of Week
        trades_df['day_of_week'] = trades_df['entry_time'].dt.day_name()
        daily_pnl = trades_df.groupby('day_of_week')['monetary_pnl'].sum().reindex([
            'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], fill_value=0)
            
        fig_day = px.bar(daily_pnl, 
                         x=daily_pnl.index, 
                         y=daily_pnl.values, 
                         title="P&L by Day of Week")
        fig_day.update_traces(marker_color=['#4CAF50' if x > 0 else '#F44336' for x in daily_pnl.values])
        fig_day.update_layout(
            template="plotly_dark",
            height=400,
            paper_bgcolor='#000000',
            plot_bgcolor='#0a0a0a',
            xaxis_title="Day",
            yaxis_title="P&L ($)"
        )
        html += f'<div class="chart-box"><h3>P&L by Day of Week</h3>{fig_day.to_html(full_html=False, include_plotlyjs=False)}</div>'
        
        return html

    @staticmethod
    def get_strategy_code(strategy):
        """Retrieves the source code of the strategy function"""
        try:
            if hasattr(TradingStrategies, strategy.__name__):
                source = inspect.getsource(strategy)
                return source
            return "Source code not found."
        except Exception as e:
            return f"Error retrieving source code: {e}"

    @staticmethod
    def highlight_code(code):
        """Enhanced syntax highlighting for HTML report"""
        keywords = ['def', 'return', 'if', 'elif', 'else', 'for', 'in', 'while', 'import', 
                   'from', 'class', 'self', 'and', 'or', 'not', 'try', 'except', 'break', 
                   'continue', 'with', 'as', 'lambda', 'None', 'True', 'False', 'pass', 'raise']
        
        code = code.replace('<', '&lt;').replace('>', '&gt;')
        
        # Highlight strings
        code = re.sub(r"(\".*?\"|\'.*?\')", r'<span class="code-string">\1</span>', code)
        
        # Highlight comments
        lines = code.split('\n')
        highlighted_lines = []
        for line in lines:
            if line.strip().startswith('#'):
                highlighted_lines.append(f'<span class="code-comment">{line}</span>')
            else:
                # Highlight keywords
                for keyword in keywords:
                    line = re.sub(r'\b' + keyword + r'\b', f'<span class="code-keyword">{keyword}</span>', line)
                # Highlight numbers
                line = re.sub(r'\b(\d+\.?\d*)\b', r'<span class="code-number">\1</span>', line)
                highlighted_lines.append(line)
        
        return '\n'.join(highlighted_lines)

# ======================================================================
# 7. CUSTOMTKINTER UI - DARK EDITION WITH CURVED CORNERS
# ======================================================================
class BacktesterUI:
    def __init__(self, master):
        self.master = master
        master.title("🚀 AlgoHaus Backtester v5.0 - PARQUET Edition")
        
        # Set window properties
        master.geometry("1400x900")
        master.minsize(1200, 700)
        
        # Default settings
        self.data_folder = pathlib.Path.cwd() / "data" 
        self.df = None
        self.selected_pair = tk.StringVar(master, value="EUR/USD")
        self.selected_timeframe = tk.StringVar(master, value="1hr")
        self.selected_strategy = tk.StringVar(master, value="vwap_crossover_strategy")
        self.initial_balance = tk.DoubleVar(master, value=10000.0)
        self.unit_size = tk.IntVar(master, value=10000)
        self.leverage = tk.IntVar(master, value=50)
        self.sl_pips = tk.IntVar(master, value=30)
        self.tp_pips = tk.IntVar(master, value=60)
        self.risk_percent = tk.DoubleVar(master, value=1.0)
        
        # Date variables
        today = date.today()
        self.end_date_var = tk.StringVar(master, value=today.strftime("%Y-%m-%d"))
        self.start_date_var = tk.StringVar(master, value=(today - timedelta(days=365)).strftime("%Y-%m-%d"))
        
        # Status/Result variables
        self.status_text = tk.StringVar(master, value="Ready. (PARQUET Version)")
        self.metrics_data = {}
        self.trades_df = pd.DataFrame()
        
        self.setup_ui()

    def setup_ui(self):
        # Main container with padding
        main_container = ctk.CTkFrame(self.master, corner_radius=0)
        main_container.pack(fill='both', expand=True)
        
        # Create a two-column layout
        left_panel = ctk.CTkFrame(main_container, corner_radius=15, width=450)
        left_panel.pack(side='left', fill='both', padx=(20, 10), pady=20)
        left_panel.pack_propagate(False)
        
        right_panel = ctk.CTkFrame(main_container, corner_radius=15)
        right_panel.pack(side='right', fill='both', expand=True, padx=(10, 20), pady=20)
        
        # ============= LEFT PANEL =============
        # Title with gradient-like effect
        title_frame = ctk.CTkFrame(left_panel, corner_radius=10, fg_color="#1a1a1a")
        title_frame.pack(fill='x', padx=20, pady=(20, 15))
        
        title_label = ctk.CTkLabel(
            title_frame,
            text="⚡ ALGOHAUS BACKTESTER - PARQUET",
            font=ctk.CTkFont(family="Helvetica", size=18, weight="bold"),
            text_color="#1976D2"
        )
        title_label.pack(pady=15)
        
        subtitle_label = ctk.CTkLabel(
            title_frame,
            text="by GhostOkaamiii (Parquet Version)",
            font=ctk.CTkFont(family="Helvetica", size=12),
            text_color="#888888"
        )
        subtitle_label.pack(pady=(0, 15))
        
        # Scrollable frame for controls
        controls_scroll = ctk.CTkScrollableFrame(left_panel, corner_radius=10, fg_color="transparent")
        controls_scroll.pack(fill='both', expand=True, padx=10, pady=(0, 10))
        
        # Configuration Section
        config_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        config_frame.pack(fill='x', pady=(0, 15))
        
        config_title = ctk.CTkLabel(
            config_frame,
            text="⚙️ Configuration",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#1976D2",
            anchor="w"
        )
        config_title.pack(fill='x', padx=15, pady=(10, 5))
        
        # Data folder selection
        folder_frame = ctk.CTkFrame(config_frame, fg_color="transparent")
        folder_frame.pack(fill='x', padx=15, pady=5)
        
        ctk.CTkLabel(
            folder_frame,
            text="Data Folder:",
            font=ctk.CTkFont(family="Helvetica", size=12),
            anchor="w",
            width=100
        ).pack(side='left')
        
        self.folder_label = ctk.CTkLabel(
            folder_frame,
            text=str(self.data_folder)[:25] + "...",
            font=ctk.CTkFont(family="Helvetica", size=12),
            text_color="#888888",
            anchor="w"
        )
        self.folder_label.pack(side='left', expand=True, fill='x', padx=10)
        
        ctk.CTkButton(
            folder_frame,
            text="Browse",
            command=self.select_data_folder,
            width=70,
            height=28,
            corner_radius=6,
            font=ctk.CTkFont(family="Helvetica", size=12),
            fg_color="#434446",
            hover_color="#595C5E"
        ).pack(side='right')
        
        # Pair selection
        self.create_input_field(config_frame, "Trading Pair:", self.selected_pair, 
                               is_combobox=True, values=list(ForexCalculator.PIP_VALUES.keys()))
        
        # Timeframe
        self.create_input_field(config_frame, "Timeframe:", self.selected_timeframe,
                               is_combobox=True, values=["1min", "5min", "15min", "1hr", "1Day"])
        
        # Date inputs
        self.create_input_field(config_frame, "Start Date:", self.start_date_var)
        self.create_input_field(config_frame, "End Date:", self.end_date_var)
        
        # Strategy Section
        strategy_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        strategy_frame.pack(fill='x', pady=(0, 15))
        
        strategy_title = ctk.CTkLabel(
            strategy_frame,
            text="💡 Strategy & Risk",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#1976D2",
            anchor="w"
        )
        strategy_title.pack(fill='x', padx=15, pady=(10, 5))
        
        # Strategy selection
        strategies = [name for name, obj in inspect.getmembers(TradingStrategies) 
                     if inspect.isfunction(obj)]
        self.create_input_field(strategy_frame, "Strategy:", self.selected_strategy,
                               is_combobox=True, values=strategies)
        
        # Risk parameters
        self.create_input_field(strategy_frame, "Stop Loss (Pips):", self.sl_pips)
        self.create_input_field(strategy_frame, "Take Profit (Pips):", self.tp_pips)
        
        # Account Section
        account_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        account_frame.pack(fill='x', pady=(0, 15))
        
        account_title = ctk.CTkLabel(
            account_frame,
            text="🏦 Account Settings",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#1976D2",
            anchor="w"
        )
        account_title.pack(fill='x', padx=15, pady=(10, 5))
        
        self.create_input_field(account_frame, "Initial Balance ($):", self.initial_balance)
        self.create_input_field(account_frame, "Unit Size:", self.unit_size)
        self.create_input_field(account_frame, "Leverage:", self.leverage,
                               is_combobox=True, values=[str(x) for x in ForexCalculator.LEVERAGE_OPTIONS])
        
        # Run Button
        run_button = ctk.CTkButton(
            left_panel,
            text="🚀 RUN BACKTEST",
            command=self.start_backtest_thread,
            height=45,
            corner_radius=8,
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            fg_color="#1976D2",
            hover_color="#1565C0"
        )
        run_button.pack(fill='x', padx=20, pady=(10, 20))
        
        # ============= RIGHT PANEL =============
        # Summary Section
        summary_frame = ctk.CTkFrame(right_panel, corner_radius=10)
        summary_frame.pack(fill='x', padx=15, pady=(15, 10))
        
        summary_title = ctk.CTkLabel(
            summary_frame,
            text="📈 Summary",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#1976D2",
            anchor="w"
        )
        summary_title.pack(fill='x', padx=15, pady=(10, 5))
        
        self.summary_textbox = ctk.CTkTextbox(
            summary_frame,
            height=150,
            corner_radius=8,
            font=ctk.CTkFont(family="Courier New", size=12),
            fg_color="#1a1a1a",
            text_color="#4CAF50"
        )
        self.summary_textbox.pack(fill='both', padx=15, pady=(5, 15))
        
        # Metrics Section with custom scrollable view
        metrics_frame = ctk.CTkFrame(right_panel, corner_radius=10)
        metrics_frame.pack(fill='both', expand=True, padx=15, pady=(0, 10))
        
        metrics_title = ctk.CTkLabel(
            metrics_frame,
            text="📊 Detailed Metrics",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#1976D2",
            anchor="w"
        )
        metrics_title.pack(fill='x', padx=15, pady=(10, 5))
        
        # Create scrollable frame for metrics
        self.metrics_scroll = ctk.CTkScrollableFrame(
            metrics_frame,
            corner_radius=8,
            fg_color="#1a1a1a"
        )
        self.metrics_scroll.pack(fill='both', expand=True, padx=15, pady=(5, 15))
        
        # Report Generation Button
        self.report_button = ctk.CTkButton(
            right_panel,
            text="📋 Generate HTML Report",
            command=self.generate_report,
            height=40,
            corner_radius=8,
            font=ctk.CTkFont(family="Helvetica", size=13, weight="bold"),
            fg_color="#434446",
            hover_color="#595C5E",
            state="disabled"
        )
        self.report_button.pack(fill='x', padx=15, pady=(0, 15))
        
        # Status Bar
        status_frame = ctk.CTkFrame(self.master, corner_radius=0, height=35, fg_color="#111111")
        status_frame.pack(side='bottom', fill='x')
        
        self.status_label = ctk.CTkLabel(
            status_frame,
            textvariable=self.status_text,
            font=ctk.CTkFont(family="Helvetica", size=11),
            text_color="#888888",
            anchor="w"
        )
        self.status_label.pack(side='left', padx=20, pady=8)

    def create_input_field(self, parent, label_text, variable, is_combobox=False, values=None):
        """Helper method to create consistent input fields"""
        frame = ctk.CTkFrame(parent, fg_color="transparent")
        frame.pack(fill='x', padx=15, pady=5)
        
        label = ctk.CTkLabel(
            frame,
            text=label_text,
            font=ctk.CTkFont(family="Helvetica", size=12),
            anchor="w",
            width=120
        )
        label.pack(side='left')
        
        if is_combobox:
            widget = ctk.CTkComboBox(
                frame,
                variable=variable,
                values=values or [],
                corner_radius=6,
                font=ctk.CTkFont(family="Helvetica", size=12),
                dropdown_font=ctk.CTkFont(family="Helvetica", size=11),
                button_color="#434446",
                button_hover_color="#595C5E",
                fg_color="#1a1a1a",
                border_width=1,
                border_color="#333333"
            )
        else:
            widget = ctk.CTkEntry(
                frame,
                textvariable=variable,
                corner_radius=6,
                font=ctk.CTkFont(family="Helvetica", size=12),
                fg_color="#1a1a1a",
                border_width=1,
                border_color="#333333"
            )
        widget.pack(side='left', expand=True, fill='x', padx=(10, 0))
        
        return widget

    def update_status(self, text, color='#888888'):
        """Update status with color coding"""
        self.status_text.set(text)
        self.status_label.configure(text_color=color)

    def select_data_folder(self):
        new_folder = filedialog.askdirectory(title="Select Data Folder (with PARQUET files)")
        if new_folder:
            self.data_folder = pathlib.Path(new_folder)
            folder_text = str(self.data_folder)
            if len(folder_text) > 25:
                folder_text = "..." + folder_text[-22:]
            self.folder_label.configure(text=folder_text)
            self.update_status(f"Data folder set to: {self.data_folder}", '#4CAF50')

    def start_backtest_thread(self):
        self.update_status("Loading PARQUET data and starting backtest...", '#FFC107')
        
        # Clear previous results
        self.summary_textbox.delete("0.0", "end")
        self.trades_df = pd.DataFrame()
        
        # Clear metrics display
        for widget in self.metrics_scroll.winfo_children():
            widget.destroy()
        
        # Disable report button
        self.report_button.configure(state="disabled")
        
        # Validate inputs
        try:
            start_date = datetime.strptime(self.start_date_var.get(), "%Y-%m-%d")
            end_date = datetime.strptime(self.end_date_var.get(), "%Y-%m-%d")
            sl_pips = self.sl_pips.get()
            tp_pips = self.tp_pips.get()
            balance = self.initial_balance.get()
            units = self.unit_size.get()
            leverage = self.leverage.get()
            
            if start_date >= end_date:
                raise ValueError("Start date must be before End date.")
            if sl_pips <= 0 or tp_pips <= 0 or balance <= 0 or units <= 0 or leverage <= 0:
                raise ValueError("SL, TP, Balance, Unit Size, and Leverage must be positive.")

        except ValueError as e:
            messagebox.showerror("Input Error", f"Invalid input: {e}")
            self.update_status("Error: Invalid input.", '#F44336')
            return

        # Start the background thread for backtesting
        self.q = queue.Queue()
        threading.Thread(target=self.run_backtest_task, 
                        args=(start_date, end_date), 
                        daemon=True).start()
        self.master.after(100, self.check_queue)

    def run_backtest_task(self, start_date, end_date):
        try:
            pair = self.selected_pair.get()
            timeframe = self.selected_timeframe.get()
            strategy_name = self.selected_strategy.get()
            
            # 1. Load Data from PARQUET
            df, _, _ = load_pair_data(pair, self.data_folder, start_date, end_date, timeframe)
            self.df = df
            
            # 2. Get Strategy Function
            strategy_func = getattr(TradingStrategies, strategy_name)
            
            # 3. Initialize and Run Backtester
            backtester = EnhancedBacktester(
                df, 
                initial_balance=self.initial_balance.get(), 
                unit_size=self.unit_size.get(), 
                pip_value=ForexCalculator.PIP_VALUES.get(pair, 0.0001), 
                leverage=self.leverage.get()
            )
            
            summary, metrics = backtester.run_backtest(
                strategy_func, 
                self.sl_pips.get(), 
                self.tp_pips.get(), 
                pair
            )
            
            # Place results in the queue
            self.q.put(('success', summary, metrics, backtester.results))

        except Exception as e:
            logging.error(f"Backtest Thread Error: {e}")
            self.q.put(('error', str(e)))

    def check_queue(self):
        try:
            result_type, *data = self.q.get_nowait()
            
            if result_type == 'success':
                summary, metrics, trades_df = data
                self.update_results_ui(summary, metrics, trades_df)
                self.update_status("✅ Backtest completed successfully! (PARQUET data)", '#4CAF50')
            elif result_type == 'error':
                error_msg = data[0]
                messagebox.showerror("Backtest Error", error_msg)
                self.update_status("❌ Error: Backtest failed.", '#F44336')

        except queue.Empty:
            self.master.after(100, self.check_queue)

    def update_results_ui(self, summary, metrics, trades_df):
        # Save trades DataFrame
        self.trades_df = trades_df
        
        # Update Summary
        self.summary_textbox.delete("0.0", "end")
        self.summary_textbox.insert("0.0", summary)
        
        # Update Metrics Display
        self.metrics_data = metrics
        
        # Clear previous metrics
        for widget in self.metrics_scroll.winfo_children():
            widget.destroy()
        
        # Create metrics cards
        row_frame = None
        for i, (key, value) in enumerate(metrics.items()):
            if i % 2 == 0:
                row_frame = ctk.CTkFrame(self.metrics_scroll, fg_color="transparent")
                row_frame.pack(fill='x', pady=5)
            
            # Create metric card
            card = ctk.CTkFrame(row_frame, corner_radius=8, fg_color="#252525", width=250)
            card.pack(side='left', expand=True, fill='x', padx=5)
            card.pack_propagate(False)
            
            # Format key for display
            display_key = key.replace('_', ' ').replace('%', '').replace('$', '').title()
            if '$' in key:
                display_key += ' ($)'
            elif '%' in key:
                display_key += ' (%)'
            
            # Determine color based on metric
            if isinstance(value, (int, float)):
                if 'win' in key.lower() or 'profit' in key.lower() or 'return' in key.lower():
                    color = '#4CAF50' if value > 0 else '#F44336'
                elif 'loss' in key.lower() or 'drawdown' in key.lower():
                    color = '#F44336'
                else:
                    color = '#1976D2'
            else:
                color = '#888888'
            
            # Metric label
            label = ctk.CTkLabel(
                card,
                text=display_key,
                font=ctk.CTkFont(family="Helvetica", size=11),
                text_color="#888888",
                anchor="w"
            )
            label.pack(fill='x', padx=10, pady=(8, 2))
            
            # Metric value
            value_str = f"{value:,.2f}" if isinstance(value, float) else str(value)
            value_label = ctk.CTkLabel(
                card,
                text=value_str,
                font=ctk.CTkFont(family="Helvetica", size=16, weight="bold"),
                text_color=color,
                anchor="w"
            )
            value_label.pack(fill='x', padx=10, pady=(2, 8))
        
        # Enable report button
        self.report_button.configure(state="normal")

    def generate_report(self):
        if self.trades_df.empty:
            messagebox.showwarning("Report Warning", 
                                  "No trades found. Please run a successful backtest first.")
            self.update_status("⚠️ No trades to report.", '#FFC107')
            return

        try:
            self.update_status("Generating HTML report (PARQUET data)...", '#FFC107')
            
            report_path = AdvancedHTMLReportGenerator.generate_dashboard_report(
                self.metrics_data, 
                self.trades_df, 
                getattr(TradingStrategies, self.selected_strategy.get()), 
                self.selected_timeframe.get(), 
                self.selected_pair.get(), 
                self.initial_balance.get(),
                self.unit_size.get(),
                df=self.df
            )
            
            messagebox.showinfo("Report Generated", 
                              f"HTML Report saved to:\n{report_path}")
            webbrowser.open_new_tab('file://' + os.path.realpath(report_path))
            self.update_status("📊 HTML report generated and opened (PARQUET data).", '#4CAF50')
        
        except Exception as e:
            messagebox.showerror("Report Error", f"Failed to generate report: {e}")
            self.update_status("❌ Error generating report.", '#F44336')

# ======================================================================
# 8. MAIN ENTRY POINT
# ======================================================================
if __name__ == '__main__':
    # Create the app
    app = ctk.CTk()
    
    # Center window on screen
    app.update_idletasks()
    width = 1400
    height = 900
    x = (app.winfo_screenwidth() // 2) - (width // 2)
    y = (app.winfo_screenheight() // 2) - (height // 2)
    app.geometry(f'{width}x{height}+{x}+{y}')
    
    # Initialize the UI
    backtester = BacktesterUI(app)
    
    # Run the app
    app.mainloop()

ERROR: Backtest Thread Error: No PARQUET file found for EUR/USD
INFO: Loading parquet file: C:\Users\Wolfrank\Desktop\AlgoHaus\OandaHistoricalData\1MinCharts\EUR_USD\EUR_USD.parquet
C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_16532\2564924156.py:236: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.resample(rule).agg({
INFO: Loaded 5065 rows from parquet file
INFO: Dataframe size: 5065 bars.
INFO: Strategy generated: 1016 potential trades.
c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behav

In [2]:
#CLAUDE VERSION 
#version 2 

# AlgoHaus Backtester v5.0 - Complete Dark Theme Edition with KPI Dashboard
# Features: Pure Dark Theme UI, Interactive KPI Dashboard with Charts, Advanced Metrics, AI Analysis
# Modified to read PARQUET files with comprehensive visualizations

import customtkinter as ctk
import tkinter as tk
from tkinter import messagebox, filedialog
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import webbrowser
import os
import tempfile
import inspect
import pathlib
import threading
import queue
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import re 
import logging

# Configure CustomTkinter - COMPLETE DARK THEME
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("green")

# Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# ======================================================================
# 1. MARGIN AND PIP CALCULATIONS
# ======================================================================
class ForexCalculator:
    """Handle all forex calculations including margins, pip values, and position sizing"""
   
    LEVERAGE_OPTIONS = [1, 10, 20, 30, 50, 100, 200, 500]
   
    PIP_VALUES = {
        'EUR/USD': 0.0001, 'GBP/USD': 0.0001, 'USD/JPY': 0.01,
        'USD/CHF': 0.0001, 'USD/CAD': 0.0001, 'AUD/USD': 0.0001,
        'NZD/USD': 0.0001, 'EUR/JPY': 0.01, 'GBP/JPY': 0.01,
        'AUD/JPY': 0.01, 'EUR/GBP': 0.0001, 'EUR/CHF': 0.0001,
        'GBP/CHF': 0.0001, 'CHF/JPY': 0.01, 'CAD/JPY': 0.01
    }
    
    @staticmethod
    def get_max_units_per_100usd(leverage=50):
        return 100 * leverage 
   
    @staticmethod
    def calculate_pip_value_in_usd(pair, unit_size, current_price, conversion_rate=1.0):
        pip_size = ForexCalculator.PIP_VALUES.get(pair, 0.0001) 
        
        if pair.endswith('/USD'):
            pip_value = pip_size * unit_size
        elif pair.startswith('USD/'):
            pip_value = (pip_size / current_price) * unit_size
        else:
            pip_value_in_base = pip_size * unit_size
            pip_value = pip_value_in_base * conversion_rate 
        
        return pip_value
   
    @staticmethod
    def calculate_margin_required(pair, unit_size, current_price, leverage, conversion_rate=1.0):
        if pair.endswith('/USD'):
            position_value = unit_size * current_price
        elif pair.startswith('USD/'):
            position_value = unit_size
        else:
            position_value = unit_size * conversion_rate 
        
        margin_required = position_value / leverage
        return margin_required

# ======================================================================
# 2. DATA LOADING FUNCTIONS
# ======================================================================
def load_pair_data(pair_name: str, base_folder: pathlib.Path, start_date: datetime, end_date: datetime, timeframe: str):
    pair_clean = pair_name.replace('/', '_')
    subfolder = base_folder / pair_clean
    parquet_files = []
   
    if subfolder.is_dir():
        parquet_files = list(subfolder.glob("*.parquet"))
   
    if not parquet_files:
        parquet_files = [f for f in base_folder.glob("*.parquet") if pair_clean.lower() in f.name.lower()]
   
    if not parquet_files:
        raise FileNotFoundError(f"No PARQUET file found for {pair_name}")
   
    parquet_path = parquet_files[0]
    df = pd.read_parquet(parquet_path, engine='pyarrow')
    
    if df.empty:
        raise ValueError("PARQUET file is empty")
   
    cols_lower = [c.strip().lower() for c in df.columns]
    col_map = {
        'datetime': ['datetime', 'date', 'time', 'timestamp', 'date_time', 'index'],
        'open': ['open', 'o', 'bid', 'ask', 'price'],
        'high': ['high', 'h', 'max'],
        'low': ['low', 'l', 'min'],
        'close': ['close', 'c', 'last', 'price'],
        'volume': ['volume', 'vol', 'v', 'tick_volume', 'size']
    }
   
    rename = {}
    for target, aliases in col_map.items():
        for alias in aliases:
            if any(alias in col for col in cols_lower):
                orig = next(col for col in df.columns if alias in col.lower())
                rename[orig] = target
                break
        else:
            if target != 'volume':
                raise KeyError(f"Column for '{target}' not found")
   
    df = df.rename(columns=rename)
    
    if 'volume' not in df.columns:
        df['volume'] = 1000
   
    for col in ['open', 'high', 'low', 'close', 'volume']:
        if col in df.columns:
            df[col] = df[col].astype('float32')
   
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', utc=True)
    df = df.dropna(subset=['datetime'])
    df['datetime'] = df['datetime'].dt.tz_localize(None)
    
    df = df.drop_duplicates(subset='datetime', keep='first')
    df = df.sort_values('datetime')
   
    actual_start = df['datetime'].min().date()
    actual_end = df['datetime'].max().date()
   
    df = df.set_index('datetime')
    user_start = max(pd.Timestamp(start_date.date()), df.index.min())
    user_end = min(pd.Timestamp(end_date.date()) + pd.Timedelta(hours=23, minutes=59, seconds=59), df.index.max())
    df = df.loc[user_start:user_end].copy()
   
    if df.empty:
        raise ValueError(f"No data in range {start_date.date()} to {end_date.date()}")
   
    if timeframe != '1min':
        rule = {'5min': '5T', '15min': '15T', '1hr': '1H', '1Day': '1D'}.get(timeframe, '1T')
        df = df.resample(rule).agg({
            'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'
        }).dropna()
   
    df = df.reset_index()
    df['date'] = df['datetime'].dt.date
   
    daily = df.groupby('date').agg({
        'high': 'max', 'low': 'min', 'close': 'last'
    })
    daily.columns = ['day_high', 'day_low', 'day_close']
    daily['prev_high'] = daily['day_high'].shift(1)
    daily['prev_low'] = daily['day_low'].shift(1)
    daily['prev_close'] = daily['day_close'].shift(1)
   
    daily = daily.reset_index()
    df = pd.merge(df, daily[['date', 'prev_high', 'prev_low', 'prev_close']], on='date', how='left')
    df[['prev_high', 'prev_low', 'prev_close']] = df[['prev_high', 'prev_low', 'prev_close']].ffill()
   
    return df, actual_start, actual_end

# ======================================================================
# 3. TRADING STRATEGIES
# ======================================================================
class TradingStrategies:
    @staticmethod
    def vwap_crossover_strategy(df, sl_pips, tp_pips, pip_value):
        df = df.copy()
        df['tpv'] = df['volume'] * (df['high'] + df['low'] + df['close']) / 3
        df['cumvol'] = df.groupby('date')['volume'].cumsum()
        df['cumtpv'] = df.groupby('date')['tpv'].cumsum()
        df['vwap'] = df['cumtpv'] / df['cumvol']
        
        df['prev_close'] = df['close'].shift(1)
        df['prev_vwap'] = df['vwap'].shift(1)
        
        df['signal'] = np.where(
            (df['prev_close'] <= df['prev_vwap']) & (df['close'] > df['vwap']), 'BUY',
            np.where((df['prev_close'] >= df['prev_vwap']) & (df['close'] < df['vwap']), 'SELL', None)
        )
        
        entries = df[df['signal'].notna()].copy()
        trades = []
        
        for idx, row in entries.iterrows():
            remaining_data = df[df.index > idx].reset_index(drop=True)
            if len(remaining_data) > 0:
                trades.append({
                    'datetime': row['datetime'],
                    'entry_price': row['close'],
                    'signal': row['signal'],
                    'stop_loss': sl_pips * pip_value,
                    'take_profit': tp_pips * pip_value,
                    'day_data': remaining_data
                })
        
        return trades
    
    @staticmethod
    def opening_range_strategy(df, sl_pips, tp_pips, pip_value):
        df = df.copy()
        trades = []
        
        for date in df['date'].unique():
            day_data = df[df['date'] == date].reset_index(drop=True)
            if len(day_data) < 31: 
                continue
            
            opening_range = day_data.iloc[:30]
            or_high = opening_range['high'].max()
            or_low = opening_range['low'].min()
            or_mid = (or_high + or_low) / 2
            
            if len(day_data) > 30:
                entry_bar = day_data.iloc[30]
                signal = 'BUY' if entry_bar['close'] > or_mid else 'SELL'
                
                trades.append({
                    'datetime': entry_bar['datetime'],
                    'entry_price': entry_bar['close'],
                    'signal': signal,
                    'stop_loss': sl_pips * pip_value,
                    'take_profit': tp_pips * pip_value,
                    'day_data': day_data[31:].reset_index(drop=True)
                })
        
        return trades

# ======================================================================
# 4. ENHANCED BACKTESTER
# ======================================================================
class EnhancedBacktester:
    def __init__(self, df, initial_balance=10000, unit_size=10000, pip_value=0.0001, leverage=50):
        self.df = df
        self.initial_balance = initial_balance
        self.unit_size = unit_size
        self.pip_value = pip_value
        self.leverage = leverage
        self.results = None
        
    def run_backtest(self, strategy_func, sl_pips, tp_pips, pair_name):
        trades = strategy_func(self.df, sl_pips, tp_pips, self.pip_value)
        
        if not trades:
            self.results = pd.DataFrame()
            return "No trades generated.", {}
            
        results = []
        current_balance = self.initial_balance
        trade_number = 1
        
        for t in trades:
            entry_price = t['entry_price']
            signal = t['signal']
            bars = t['day_data']
            
            margin_required = ForexCalculator.calculate_margin_required(
                pair_name, self.unit_size, entry_price, self.leverage, 1.0
            )
            
            if margin_required > current_balance * 0.5:
                continue
            
            pip_value_usd = ForexCalculator.calculate_pip_value_in_usd(
                pair_name, self.unit_size, entry_price, 1.0
            )
            
            if signal == 'BUY':
                stop_level = entry_price - (sl_pips * self.pip_value)
                take_level = entry_price + (tp_pips * self.pip_value)
            else:
                stop_level = entry_price + (sl_pips * self.pip_value)
                take_level = entry_price - (tp_pips * self.pip_value)
            
            exit_idx = len(bars) - 1
            exit_reason = 'Timeout'
            exit_price = bars.iloc[-1]['close'] if len(bars) > 0 else entry_price
            
            for i, bar in bars.iterrows():
                if signal == 'BUY':
                    if bar['low'] <= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['high'] >= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
                else:
                    if bar['high'] >= stop_level:
                        exit_idx = i
                        exit_price = stop_level
                        exit_reason = 'SL'
                        break
                    elif bar['low'] <= take_level:
                        exit_idx = i
                        exit_price = take_level
                        exit_reason = 'TP'
                        break
            
            if signal == 'BUY':
                pips_pnl = (exit_price - entry_price) / self.pip_value
            else:
                pips_pnl = (entry_price - exit_price) / self.pip_value
            
            monetary_pnl = pips_pnl * pip_value_usd
            entry_time = t['datetime']
            
            if exit_idx < len(bars):
                exit_time = bars.iloc[exit_idx]['datetime']
            else:
                exit_time = bars.iloc[-1]['datetime'] if len(bars) > 0 else entry_time
            
            time_in_trade = exit_time - entry_time
            hours_in_trade = time_in_trade.total_seconds() / 3600
            current_balance += monetary_pnl
            
            results.append({
                'trade_number': f"{trade_number:05d}",
                'entry_time': entry_time,
                'exit_time': exit_time,
                'time_in_trade_hours': round(hours_in_trade, 2),
                'signal': signal,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'exit_reason': exit_reason,
                'pips_pnl': round(pips_pnl, 2),
                'monetary_pnl': round(monetary_pnl, 2),
                'unit_size': self.unit_size,
                'margin_used': round(margin_required, 2),
                'balance': round(current_balance, 2),
                'pip_value_usd': round(pip_value_usd, 4)
            })
            
            trade_number += 1
        
        self.results = pd.DataFrame(results)
        
        if not self.results.empty:
            total_pnl = self.results['monetary_pnl'].sum()
            total_pips = self.results['pips_pnl'].sum()
            win_rate = (self.results['pips_pnl'] > 0).mean() * 100
            
            summary = (f"TRADES: {len(self.results)}\n"
                      f"WIN RATE: {win_rate:.1f}%\n"
                      f"P&L: ${total_pnl:,.2f}\n"
                      f"PIPS: {total_pips:,.1f}\n"
                      f"FINAL BALANCE: ${current_balance:,.2f}")
            
            metrics = self.calculate_metrics()
        else:
            summary = "No trades executed"
            metrics = {}
        
        return summary, metrics
    
    def calculate_metrics(self):
        trades_df = self.results
        if trades_df.empty:
            return {}
        
        total_trades = len(trades_df)
        winning_trades = (trades_df['monetary_pnl'] > 0).sum()
        losing_trades = (trades_df['monetary_pnl'] < 0).sum()
        win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0
        
        total_pnl = trades_df['monetary_pnl'].sum()
        total_pips = trades_df['pips_pnl'].sum()
        avg_win = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl'].mean() if winning_trades > 0 else 0
        avg_loss = abs(trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl'].mean()) if losing_trades > 0 else 0
        
        gross_profit = trades_df[trades_df['monetary_pnl'] > 0]['monetary_pnl'].sum()
        gross_loss = abs(trades_df[trades_df['monetary_pnl'] < 0]['monetary_pnl'].sum())
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
        
        final_balance = trades_df['balance'].iloc[-1] if not trades_df.empty else self.initial_balance
        total_return = ((final_balance - self.initial_balance) / self.initial_balance) * 100
        
        equity_curve = self.initial_balance + trades_df['monetary_pnl'].cumsum()
        cummax = equity_curve.expanding().max()
        drawdown = (equity_curve - cummax) / cummax
        max_drawdown_pct = drawdown.min() * 100
        
        if len(trades_df) > 1:
            daily_returns = trades_df.groupby(trades_df['entry_time'].dt.date)['monetary_pnl'].sum()
            daily_returns_pct = daily_returns / self.initial_balance
            sharpe = (daily_returns_pct.mean() * 252) / (daily_returns_pct.std() * np.sqrt(252)) if daily_returns_pct.std() > 0 else 0
            
            negative_returns = daily_returns_pct[daily_returns_pct < 0]
            downside_std = negative_returns.std() if len(negative_returns) > 0 else 0
            sortino = (daily_returns_pct.mean() * 252) / (downside_std * np.sqrt(252)) if downside_std > 0 else 0
        else:
            sharpe = 0
            sortino = 0
        
        return {
            'total_trades': total_trades,
            'winning_trades': winning_trades,
            'losing_trades': losing_trades,
            'win_rate_%': round(win_rate, 2),
            'total_pnl_$': round(total_pnl, 2),
            'total_pips': round(total_pips, 1),
            'avg_win_$': round(avg_win, 2),
            'avg_loss_$': round(avg_loss, 2),
            'profit_factor': round(profit_factor, 2),
            'max_drawdown_%': round(max_drawdown_pct, 2),
            'sharpe_ratio': round(sharpe, 2),
            'sortino_ratio': round(sortino, 2),
            'total_return_%': round(total_return, 2),
            'final_balance_$': round(final_balance, 2)
        }

# ======================================================================
# 5. KPI DASHBOARD GENERATOR
# ======================================================================
class KPIDashboard:
    @staticmethod
    def generate_kpi_charts(metrics, trades_df):
        charts_html = ""
        
        charts_html += KPIDashboard.create_gauge_charts(metrics)
        charts_html += KPIDashboard.create_pnl_pie_chart(trades_df)
        charts_html += KPIDashboard.create_performance_heatmap(trades_df)
        charts_html += KPIDashboard.create_risk_radar_chart(metrics)
        charts_html += KPIDashboard.create_monthly_performance_bar(trades_df)
        charts_html += KPIDashboard.create_drawdown_line_chart(trades_df, metrics)
        
        return charts_html
    
    @staticmethod
    def create_gauge_charts(metrics):
        kpis = {
            'Sharpe Ratio': {'value': metrics.get('sharpe_ratio', 0), 'min': -3, 'max': 3},
            'Sortino Ratio': {'value': metrics.get('sortino_ratio', 0), 'min': -3, 'max': 3},
            'Win Rate': {'value': metrics.get('win_rate_%', 0), 'min': 0, 'max': 100},
            'Profit Factor': {'value': metrics.get('profit_factor', 0), 'min': 0, 'max': 3}
        }
        
        html = '<div class="kpi-gauges-container">'
        
        for kpi_name, kpi_data in kpis.items():
            value = kpi_data['value']
            min_val = kpi_data['min']
            max_val = kpi_data['max']
            
            if kpi_name in ['Sharpe Ratio', 'Sortino Ratio']:
                color = '#ff4757' if value < 0 else '#32ff7e' if value > 1 else '#ff9f43'
            elif kpi_name == 'Win Rate':
                color = '#ff4757' if value < 40 else '#32ff7e' if value > 60 else '#ff9f43'
            elif kpi_name == 'Profit Factor':
                color = '#ff4757' if value < 1 else '#32ff7e' if value > 1.5 else '#ff9f43'
            else:
                color = '#00ff41'
            
            fig = go.Figure(go.Indicator(
                mode = "gauge+number",
                value = value,
                domain = {'x': [0, 1], 'y': [0, 1]},
                title = {'text': kpi_name, 'font': {'color': '#ffffff', 'size': 14}},
                number = {'font': {'color': color, 'size': 20}},
                gauge = {
                    'axis': {'range': [min_val, max_val], 'tickcolor': '#333333', 'tickfont': {'color': '#ffffff'}},
                    'bar': {'color': color},
                    'bgcolor': "#000000",
                    'borderwidth': 2,
                    'bordercolor': "#333333"
                }
            ))
            
            fig.update_layout(
                template="plotly_dark",
                paper_bgcolor='#000000',
                plot_bgcolor='#000000',
                font={'color': '#ffffff'},
                height=300,
                margin=dict(l=20, r=20, t=40, b=20)
            )
            
            html += f'<div class="kpi-gauge-box"><h4 class="kpi-title">{kpi_name}</h4>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'
        
        html += '</div>'
        return html
    
    @staticmethod
    def create_pnl_pie_chart(trades_df):
        if trades_df.empty:
            return ""
        
        wins = len(trades_df[trades_df['monetary_pnl'] > 0])
        losses = len(trades_df[trades_df['monetary_pnl'] < 0])
        
        fig = go.Figure(data=[go.Pie(
            labels=['Winning Trades', 'Losing Trades'], 
            values=[wins, losses],
            hole=0.4,
            marker=dict(colors=['#00ff41', '#ff4757'], line=dict(color='#000000', width=2))
        )])
        
        fig.update_layout(
            template="plotly_dark",
            title={'text': 'Trade Distribution', 'font': {'color': '#ffffff', 'size': 16}, 'x': 0.5},
            paper_bgcolor='#000000',
            plot_bgcolor='#000000',
            height=400
        )
        
        return f'<div class="chart-box"><h3>Trade Distribution</h3>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'
    
    @staticmethod
    def create_performance_heatmap(trades_df):
        if trades_df.empty or len(trades_df) < 10:
            return ""
        
        trades_df_copy = trades_df.copy()
        trades_df_copy['hour'] = trades_df_copy['entry_time'].dt.hour
        trades_df_copy['day_name'] = trades_df_copy['entry_time'].dt.day_name()
        
        heatmap_data = trades_df_copy.groupby(['day_name', 'hour'])['monetary_pnl'].sum().reset_index()
        pivot_data = heatmap_data.pivot(index='day_name', columns='hour', values='monetary_pnl').fillna(0)
        
        fig = go.Figure(data=go.Heatmap(
            z=pivot_data.values,
            x=pivot_data.columns,
            y=pivot_data.index,
            colorscale='RdYlGn',
            zmid=0
        ))
        
        fig.update_layout(
            template="plotly_dark",
            title={'text': 'Performance Heatmap', 'font': {'color': '#ffffff', 'size': 16}, 'x': 0.5},
            paper_bgcolor='#000000',
            plot_bgcolor='#000000',
            height=400
        )
        
        return f'<div class="chart-box"><h3>Performance Heatmap</h3>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'
    
    @staticmethod
    def create_risk_radar_chart(metrics):
        categories = ['Sharpe Ratio', 'Sortino Ratio', 'Profit Factor', 'Win Rate']
        values = [
            max(0, min(1, (metrics.get('sharpe_ratio', 0) + 3) / 6)),
            max(0, min(1, (metrics.get('sortino_ratio', 0) + 3) / 6)),
            max(0, min(1, metrics.get('profit_factor', 0) / 3)),
            max(0, min(1, metrics.get('win_rate_%', 0) / 100))
        ]
        
        fig = go.Figure()
        fig.add_trace(go.Scatterpolar(
            r=values,
            theta=categories,
            fill='toself',
            line=dict(color='#00ff41'),
            fillcolor='rgba(0, 255, 65, 0.3)'
        ))
        
        fig.update_layout(
            template="plotly_dark",
            title={'text': 'Risk Profile', 'font': {'color': '#ffffff', 'size': 16}, 'x': 0.5},
            paper_bgcolor='#000000',
            plot_bgcolor='#000000',
            height=400
        )
        
        return f'<div class="chart-box"><h3>Risk Profile</h3>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'
    
    @staticmethod
    def create_monthly_performance_bar(trades_df):
        if trades_df.empty or len(trades_df) < 5:
            return ""
        
        trades_df_copy = trades_df.copy()
        trades_df_copy['month'] = trades_df_copy['entry_time'].dt.to_period('M')
        monthly_pnl = trades_df_copy.groupby('month')['monetary_pnl'].sum()
        
        colors = ['#00ff41' if x >= 0 else '#ff4757' for x in monthly_pnl.values]
        
        fig = go.Figure(data=[go.Bar(
            x=[str(m) for m in monthly_pnl.index],
            y=monthly_pnl.values,
            marker_color=colors
        )])
        
        fig.update_layout(
            template="plotly_dark",
            title={'text': 'Monthly Performance', 'font': {'color': '#ffffff', 'size': 16}, 'x': 0.5},
            paper_bgcolor='#000000',
            plot_bgcolor='#000000',
            height=400
        )
        
        return f'<div class="chart-box"><h3>Monthly Performance</h3>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'
    
    @staticmethod
    def create_drawdown_line_chart(trades_df, metrics):
        if trades_df.empty:
            return ""
        
        initial_balance = trades_df['balance'].iloc[0] - trades_df['monetary_pnl'].iloc[0]
        equity_curve = initial_balance + trades_df['monetary_pnl'].cumsum()
        cummax = equity_curve.expanding().max()
        drawdown = (equity_curve - cummax) / cummax * 100
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=trades_df['exit_time'],
            y=drawdown,
            mode='lines',
            line=dict(color='#ff4757', width=2),
            fill='tozeroy',
            fillcolor='rgba(255, 71, 87, 0.3)'
        ))
        
        fig.update_layout(
            template="plotly_dark",
            title={'text': 'Drawdown Analysis', 'font': {'color': '#ffffff', 'size': 16}, 'x': 0.5},
            paper_bgcolor='#000000',
            plot_bgcolor='#000000',
            height=400
        )
        
        return f'<div class="chart-box"><h3>Drawdown Analysis</h3>{fig.to_html(full_html=False, include_plotlyjs=False)}</div>'

# ======================================================================
# 6. HTML REPORT GENERATOR
# ======================================================================
class HTMLReportGenerator:
    @staticmethod
    def generate_report(metrics, trades_df, strategy, timeframe, pair, initial_balance, unit_size, df=None):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        kpi_dashboard_html = KPIDashboard.generate_kpi_charts(metrics, trades_df)
        
        html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>AlgoHaus Dark Report - {pair} {strategy.__name__}</title>
            <meta charset="utf-8">
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            <style>
                * {{ box-sizing: border-box; margin: 0; padding: 0; }}
                body {{ font-family: 'Helvetica', monospace; background: #000000; color: #ffffff; line-height: 1.6; }}
                
                .header {{ background: #000000; color: #ffffff; padding: 30px; border-bottom: 2px solid #00ff41; text-align: center; }}
                .header h1 {{ font-size: 24px; margin-bottom: 10px; color: #00ff41; text-shadow: 0 0 10px rgba(0, 255, 65, 0.5); }}
                .header p {{ color: #888888; font-size: 14px; }}
                
                .kpi-dashboard-section {{ background: #000000; padding: 20px; border-bottom: 1px solid #222222; }}
                .kpi-dashboard-section h2 {{ color: #00ff41; font-size: 18px; margin-bottom: 20px; font-weight: bold; }}
                
                .kpi-gauges-container {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 20px; margin-bottom: 30px; }}
                .kpi-gauge-box {{ background: #0a0a0a; border: 1px solid #222222; border-radius: 8px; padding: 15px; transition: all 0.3s ease; }}
                .kpi-gauge-box:hover {{ background: #111111; border-color: #00ff41; box-shadow: 0 0 10px rgba(0, 255, 65, 0.2); }}
                .kpi-title {{ color: #00ff41; font-size: 14px; margin-bottom: 10px; text-align: center; font-weight: bold; }}
                
                .charts-container {{ background: #000000; padding: 20px; }}
                .charts-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(500px, 1fr)); gap: 20px; }}
                .chart-box {{ background: #0a0a0a; border: 1px solid #222222; border-radius: 8px; padding: 15px; }}
                .chart-box h3 {{ color: #00ff41; font-size: 14px; margin-bottom: 15px; text-align: center; }}
                
                .metrics-section {{ background: #000000; padding: 20px; border-bottom: 1px solid #222222; }}
                .metrics-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 20px; margin-top: 20px; }}
                .metric-card {{ background: #0a0a0a; border: 1px solid #222222; border-radius: 8px; padding: 15px; }}
                .metric-label {{ color: #888888; font-size: 12px; text-transform: uppercase; margin-bottom: 5px; }}
                .metric-value {{ font-size: 24px; font-weight: bold; color: #ffffff; }}
                .metric-value.positive {{ color: #00ff41; }}
                .metric-value.negative {{ color: #ff4757; }}
                .metric-value.neutral {{ color: #ffa502; }}
                
                @media (max-width: 768px) {{
                    .metrics-grid, .charts-grid, .kpi-gauges-container {{ grid-template-columns: 1fr; }}
                }}
            </style>
        </head>
        <body>
            <div class="header">
                <h1>🚀 AlgoHaus Dark Report v5.0</h1>
                <p>📊 {pair} | 💡 {strategy.__name__} | ⏰ {timeframe} | 📅 {timestamp}</p>
            </div>
            
            <div class="kpi-dashboard-section">
                <h2>📊 Interactive KPI Dashboard</h2>
                {kpi_dashboard_html}
            </div>
            
            <div class="metrics-section">
                <h2 style="color: #00ff41; font-size: 18px; margin-bottom: 0;">📈 Performance Metrics</h2>
                {HTMLReportGenerator.generate_metrics_dashboard(metrics, initial_balance)}
            </div>
        </body>
        </html>
        """
        
        temp_dir = tempfile.gettempdir()
        report_path = os.path.join(temp_dir, f"dark_report_{pair.replace('/', '_')}_{strategy.__name__}_{timestamp}.html")
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(html)
            
        return report_path
    
    @staticmethod
    def generate_metrics_dashboard(metrics, initial_balance):
        def get_metric_class(value, metric_type):
            if metric_type == 'return':
                return 'positive' if value >= 0 else 'negative'
            elif metric_type == 'winrate':
                return 'positive' if value >= 50 else 'negative'
            elif metric_type == 'factor':
                return 'positive' if value >= 1 else 'negative'
            else:
                return 'neutral'
        
        html = f"""
        <div class="metrics-grid">
            <div class="metric-card">
                <div class="metric-label">Initial Balance</div>
                <div class="metric-value neutral">${initial_balance:,.2f}</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Final Balance</div>
                <div class="metric-value {get_metric_class(metrics.get('final_balance_$', 0) - initial_balance, 'return')}">${metrics.get('final_balance_$', 0):,.2f}</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Total Return</div>
                <div class="metric-value {get_metric_class(metrics.get('total_return_%', 0), 'return')}">{metrics.get('total_return_%', 0):+.2f}%</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Win Rate</div>
                <div class="metric-value {get_metric_class(metrics.get('win_rate_%', 0), 'winrate')}">{metrics.get('win_rate_%', 0):.1f}%</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Profit Factor</div>
                <div class="metric-value {get_metric_class(metrics.get('profit_factor', 0), 'factor')}">{metrics.get('profit_factor', 0):.2f}</div>
            </div>
            <div class="metric-card">
                <div class="metric-label">Sharpe Ratio</div>
                <div class="metric-value neutral">{metrics.get('sharpe_ratio', 0):.2f}</div>
            </div>
        </div>
        """
        return html

# ======================================================================
# 7. DARK THEME UI
# ======================================================================
class BacktesterUI:
    def __init__(self, master):
        self.master = master
        master.title("🚀 AlgoHaus Backtester v5.0 - Complete Dark Edition")
        master.geometry("1400x900")
        master.minsize(1200, 700)
        
        self.data_folder = pathlib.Path.cwd() / "data" 
        self.df = None
        self.selected_pair = tk.StringVar(master, value="EUR/USD")
        self.selected_timeframe = tk.StringVar(master, value="1hr")
        self.selected_strategy = tk.StringVar(master, value="vwap_crossover_strategy")
        self.initial_balance = tk.DoubleVar(master, value=10000.0)
        self.unit_size = tk.IntVar(master, value=10000)
        self.leverage = tk.IntVar(master, value=50)
        self.sl_pips = tk.IntVar(master, value=30)
        self.tp_pips = tk.IntVar(master, value=60)
        
        today = date.today()
        self.end_date_var = tk.StringVar(master, value=today.strftime("%Y-%m-%d"))
        self.start_date_var = tk.StringVar(master, value=(today - timedelta(days=365)).strftime("%Y-%m-%d"))
        
        self.status_text = tk.StringVar(master, value="Ready - Complete Dark Edition")
        self.metrics_data = {}
        self.trades_df = pd.DataFrame()
        
        self.setup_ui()

    def setup_ui(self):
        main_container = ctk.CTkFrame(self.master, corner_radius=0)
        main_container.pack(fill='both', expand=True)
        
        left_panel = ctk.CTkFrame(main_container, corner_radius=15, width=450)
        left_panel.pack(side='left', fill='both', padx=(20, 10), pady=20)
        left_panel.pack_propagate(False)
        
        right_panel = ctk.CTkFrame(main_container, corner_radius=15)
        right_panel.pack(side='right', fill='both', expand=True, padx=(10, 20), pady=20)
        
        # Left Panel
        title_frame = ctk.CTkFrame(left_panel, corner_radius=10, fg_color="#1a1a1a")
        title_frame.pack(fill='x', padx=20, pady=(20, 15))
        
        title_label = ctk.CTkLabel(
            title_frame,
            text="⚡ ALGOHAUS DARK EDITION",
            font=ctk.CTkFont(family="Helvetica", size=18, weight="bold"),
            text_color="#00ff41"
        )
        title_label.pack(pady=15)
        
        controls_scroll = ctk.CTkScrollableFrame(left_panel, corner_radius=10, fg_color="transparent")
        controls_scroll.pack(fill='both', expand=True, padx=10, pady=(0, 10))
        
        # Configuration
        config_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        config_frame.pack(fill='x', pady=(0, 15))
        
        config_title = ctk.CTkLabel(
            config_frame,
            text="⚙️ Configuration",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        )
        config_title.pack(fill='x', padx=15, pady=(10, 5))
        
        # Data folder
        folder_frame = ctk.CTkFrame(config_frame, fg_color="transparent")
        folder_frame.pack(fill='x', padx=15, pady=5)
        
        ctk.CTkLabel(folder_frame, text="Data Folder:", width=100).pack(side='left')
        
        self.folder_label = ctk.CTkLabel(
            folder_frame,
            text=str(self.data_folder)[:25] + "...",
            text_color="#888888",
            anchor="w"
        )
        self.folder_label.pack(side='left', expand=True, fill='x', padx=10)
        
        ctk.CTkButton(
            folder_frame,
            text="Browse",
            command=self.select_data_folder,
            width=70,
            height=28
        ).pack(side='right')
        
        # Input fields
        self.create_input_field(config_frame, "Trading Pair:", self.selected_pair, 
                               is_combobox=True, values=list(ForexCalculator.PIP_VALUES.keys()))
        self.create_input_field(config_frame, "Timeframe:", self.selected_timeframe,
                               is_combobox=True, values=["1min", "5min", "15min", "1hr", "1Day"])
        self.create_input_field(config_frame, "Start Date:", self.start_date_var)
        self.create_input_field(config_frame, "End Date:", self.end_date_var)
        
        # Strategy section
        strategy_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        strategy_frame.pack(fill='x', pady=(0, 15))
        
        strategy_title = ctk.CTkLabel(
            strategy_frame,
            text="💡 Strategy & Risk",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        )
        strategy_title.pack(fill='x', padx=15, pady=(10, 5))
        
        strategies = [name for name, obj in inspect.getmembers(TradingStrategies) if inspect.isfunction(obj)]
        self.create_input_field(strategy_frame, "Strategy:", self.selected_strategy,
                               is_combobox=True, values=strategies)
        self.create_input_field(strategy_frame, "Stop Loss (Pips):", self.sl_pips)
        self.create_input_field(strategy_frame, "Take Profit (Pips):", self.tp_pips)
        
        # Account section
        account_frame = ctk.CTkFrame(controls_scroll, corner_radius=10)
        account_frame.pack(fill='x', pady=(0, 15))
        
        account_title = ctk.CTkLabel(
            account_frame,
            text="🏦 Account Settings",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        )
        account_title.pack(fill='x', padx=15, pady=(10, 5))
        
        self.create_input_field(account_frame, "Initial Balance ($):", self.initial_balance)
        self.create_input_field(account_frame, "Unit Size:", self.unit_size)
        self.create_input_field(account_frame, "Leverage:", self.leverage,
                               is_combobox=True, values=[str(x) for x in ForexCalculator.LEVERAGE_OPTIONS])
        
        # Run button
        run_button = ctk.CTkButton(
            left_panel,
            text="🚀 RUN BACKTEST",
            command=self.start_backtest_thread,
            height=45,
            corner_radius=8,
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            fg_color="#00ff41",
            hover_color="#32ff7e",
            text_color="#000000"
        )
        run_button.pack(fill='x', padx=20, pady=(10, 20))
        
        # Right Panel
        summary_frame = ctk.CTkFrame(right_panel, corner_radius=10)
        summary_frame.pack(fill='x', padx=15, pady=(15, 10))
        
        summary_title = ctk.CTkLabel(
            summary_frame,
            text="📈 Summary",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        )
        summary_title.pack(fill='x', padx=15, pady=(10, 5))
        
        self.summary_textbox = ctk.CTkTextbox(
            summary_frame,
            height=150,
            corner_radius=8,
            font=ctk.CTkFont(family="Courier New", size=12),
            fg_color="#1a1a1a",
            text_color="#00ff41"
        )
        self.summary_textbox.pack(fill='both', padx=15, pady=(5, 15))
        
        # Metrics section
        metrics_frame = ctk.CTkFrame(right_panel, corner_radius=10)
        metrics_frame.pack(fill='both', expand=True, padx=15, pady=(0, 10))
        
        metrics_title = ctk.CTkLabel(
            metrics_frame,
            text="📊 Detailed Metrics",
            font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
            text_color="#00ff41",
            anchor="w"
        )
        metrics_title.pack(fill='x', padx=15, pady=(10, 5))
        
        self.metrics_scroll = ctk.CTkScrollableFrame(
            metrics_frame,
            corner_radius=8,
            fg_color="#1a1a1a"
        )
        self.metrics_scroll.pack(fill='both', expand=True, padx=15, pady=(5, 15))
        
        # Report button
        self.report_button = ctk.CTkButton(
            right_panel,
            text="📋 Generate Dark Report",
            command=self.generate_report,
            height=40,
            corner_radius=8,
            font=ctk.CTkFont(family="Helvetica", size=13, weight="bold"),
            fg_color="#434446",
            hover_color="#595C5E",
            state="disabled"
        )
        self.report_button.pack(fill='x', padx=15, pady=(0, 15))
        
        # Status bar
        status_frame = ctk.CTkFrame(self.master, corner_radius=0, height=35, fg_color="#111111")
        status_frame.pack(side='bottom', fill='x')
        
        self.status_label = ctk.CTkLabel(
            status_frame,
            textvariable=self.status_text,
            font=ctk.CTkFont(family="Helvetica", size=11),
            text_color="#888888",
            anchor="w"
        )
        self.status_label.pack(side='left', padx=20, pady=8)

    def create_input_field(self, parent, label_text, variable, is_combobox=False, values=None):
        frame = ctk.CTkFrame(parent, fg_color="transparent")
        frame.pack(fill='x', padx=15, pady=5)
        
        label = ctk.CTkLabel(frame, text=label_text, width=120, anchor="w")
        label.pack(side='left')
        
        if is_combobox:
            widget = ctk.CTkComboBox(frame, variable=variable, values=values or [])
        else:
            widget = ctk.CTkEntry(frame, textvariable=variable)
        widget.pack(side='left', expand=True, fill='x', padx=(10, 0))
        
        return widget

    def update_status(self, text, color='#888888'):
        self.status_text.set(text)
        self.status_label.configure(text_color=color)

    def select_data_folder(self):
        new_folder = filedialog.askdirectory(title="Select Data Folder (PARQUET files)")
        if new_folder:
            self.data_folder = pathlib.Path(new_folder)
            folder_text = str(self.data_folder)
            if len(folder_text) > 25:
                folder_text = "..." + folder_text[-22:]
            self.folder_label.configure(text=folder_text)
            self.update_status(f"Data folder set", '#00ff41')

    def start_backtest_thread(self):
        self.update_status("Starting backtest...", '#ffa502')
        
        self.summary_textbox.delete("0.0", "end")
        self.trades_df = pd.DataFrame()
        
        for widget in self.metrics_scroll.winfo_children():
            widget.destroy()
        
        self.report_button.configure(state="disabled")
        
        try:
            start_date = datetime.strptime(self.start_date_var.get(), "%Y-%m-%d")
            end_date = datetime.strptime(self.end_date_var.get(), "%Y-%m-%d")
            
            if start_date >= end_date:
                raise ValueError("Start date must be before End date.")

        except ValueError as e:
            messagebox.showerror("Input Error", f"Invalid input: {e}")
            self.update_status("Error: Invalid input.", '#ff4757')
            return

        self.q = queue.Queue()
        threading.Thread(target=self.run_backtest_task, 
                        args=(start_date, end_date), 
                        daemon=True).start()
        self.master.after(100, self.check_queue)

    def run_backtest_task(self, start_date, end_date):
        try:
            pair = self.selected_pair.get()
            timeframe = self.selected_timeframe.get()
            strategy_name = self.selected_strategy.get()
            
            df, _, _ = load_pair_data(pair, self.data_folder, start_date, end_date, timeframe)
            self.df = df
            
            strategy_func = getattr(TradingStrategies, strategy_name)
            
            backtester = EnhancedBacktester(
                df, 
                initial_balance=self.initial_balance.get(), 
                unit_size=self.unit_size.get(), 
                pip_value=ForexCalculator.PIP_VALUES.get(pair, 0.0001), 
                leverage=self.leverage.get()
            )
            
            summary, metrics = backtester.run_backtest(
                strategy_func, 
                self.sl_pips.get(), 
                self.tp_pips.get(), 
                pair
            )
            
            self.q.put(('success', summary, metrics, backtester.results))

        except Exception as e:
            self.q.put(('error', str(e)))

    def check_queue(self):
        try:
            result_type, *data = self.q.get_nowait()
            
            if result_type == 'success':
                summary, metrics, trades_df = data
                self.update_results_ui(summary, metrics, trades_df)
                self.update_status("✅ Backtest completed!", '#00ff41')
            elif result_type == 'error':
                error_msg = data[0]
                messagebox.showerror("Backtest Error", error_msg)
                self.update_status("❌ Error: Backtest failed.", '#ff4757')

        except queue.Empty:
            self.master.after(100, self.check_queue)

    def update_results_ui(self, summary, metrics, trades_df):
        self.trades_df = trades_df
        
        self.summary_textbox.delete("0.0", "end")
        self.summary_textbox.insert("0.0", summary)
        
        self.metrics_data = metrics
        
        for widget in self.metrics_scroll.winfo_children():
            widget.destroy()
        
        # Create metrics cards
        row_frame = None
        for i, (key, value) in enumerate(metrics.items()):
            if i % 2 == 0:
                row_frame = ctk.CTkFrame(self.metrics_scroll, fg_color="transparent")
                row_frame.pack(fill='x', pady=5)
            
            card = ctk.CTkFrame(row_frame, corner_radius=8, fg_color="#252525", width=250)
            card.pack(side='left', expand=True, fill='x', padx=5)
            card.pack_propagate(False)
            
            display_key = key.replace('_', ' ').title()
            
            if isinstance(value, (int, float)):
                if 'profit' in key.lower() or 'return' in key.lower():
                    color = '#00ff41' if value > 0 else '#ff4757'
                else:
                    color = '#00ff41'
            else:
                color = '#888888'
            
            label = ctk.CTkLabel(
                card,
                text=display_key,
                font=ctk.CTkFont(family="Helvetica", size=11),
                text_color="#888888",
                anchor="w"
            )
            label.pack(fill='x', padx=10, pady=(8, 2))
            
            value_str = f"{value:,.2f}" if isinstance(value, float) else str(value)
            value_label = ctk.CTkLabel(
                card,
                text=value_str,
                font=ctk.CTkFont(family="Helvetica", size=16, weight="bold"),
                text_color=color,
                anchor="w"
            )
            value_label.pack(fill='x', padx=10, pady=(2, 8))
        
        self.report_button.configure(state="normal")

    def generate_report(self):
        if self.trades_df.empty:
            messagebox.showwarning("Report Warning", "No trades found.")
            return

        try:
            self.update_status("Generating dark report...", '#ffa502')
            
            report_path = HTMLReportGenerator.generate_report(
                self.metrics_data, 
                self.trades_df, 
                getattr(TradingStrategies, self.selected_strategy.get()), 
                self.selected_timeframe.get(), 
                self.selected_pair.get(), 
                self.initial_balance.get(),
                self.unit_size.get(),
                df=self.df
            )
            
            messagebox.showinfo("Report Generated", f"Dark report saved:\n{report_path}")
            webbrowser.open_new_tab('file://' + os.path.realpath(report_path))
            self.update_status("📊 Dark report generated!", '#00ff41')
        
        except Exception as e:
            messagebox.showerror("Report Error", f"Failed to generate report: {e}")
            self.update_status("❌ Error generating report.", '#ff4757')

# ======================================================================
# 8. MAIN ENTRY POINT
# ======================================================================
if __name__ == '__main__':
    app = ctk.CTk()
    
    app.update_idletasks()
    width = 1400
    height = 900
    x = (app.winfo_screenwidth() // 2) - (width // 2)
    y = (app.winfo_screenheight() // 2) - (height // 2)
    app.geometry(f'{width}x{height}+{x}+{y}')
    
    backtester = BacktesterUI(app)
    app.mainloop()

C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_9984\2813549937.py:154: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

Exception ignored in: <function Variable.__del__ at 0x0000018542AF6AC0>
Traceback (most recent call last):
  File "c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 410, in __del__
    if self._tk.getboolean(self._tk.call("info", "exists", self._name)):
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: main thread is not in main loop
Exception ignored in: <function Variable.__del__ at 0x0000018542AF6AC0>
Traceback (most recent call last):
  File "c:\Users\Wolfrank\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 410, in __del__
    if self._tk.getboolean(self._tk.call("info", "exists", self._name)):
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: main thread is not in main loop
Exception